<a href="https://colab.research.google.com/github/henryamster/PPP-Loan-Data-Scraper-Filter/blob/main/PPP_data_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Use Google Drive?
google_drive = 'True' #@param ['True','False']

In [ ]:
#@title Import dependencies, connect google drive and set working path
# Connect to google drive
if google_drive == 'True':
    from google.colab import drive
    drive.mount('/content/drive')


#imports
import pandas as pd
import os, glob
import numpy as np
import urllib.request
import json
from os.path import exists

#set working path, create if does not exist
if google_drive == 'True':
    WORKING_PATH = '/content/drive/MyDrive/PPP'
else:
    WORKING_PATH = '/content/PPP'

if os.path.isdir(WORKING_PATH)==False:
    os.mkdir(WORKING_PATH)

#NAICS List data
NAICS = [{"NAICSCode":111110,"Description":"Soybean Farming"},{"NAICSCode":111120,"Description":"Oilseed (except Soybean) Farming"},{"NAICSCode":111130,"Description":"Dry Pea and Bean Farming"},{"NAICSCode":111140,"Description":"Wheat Farming"},{"NAICSCode":111150,"Description":"Corn Farming"},{"NAICSCode":111160,"Description":"Rice Farming"},{"NAICSCode":111191,"Description":"Oilseed and Grain Combination Farming"},{"NAICSCode":111199,"Description":"All Other Grain Farming"},{"NAICSCode":111211,"Description":"Potato Farming"},{"NAICSCode":111219,"Description":"Other Vegetable (except Potato) and Melon Farming"},{"NAICSCode":111310,"Description":"Orange Groves"},{"NAICSCode":111320,"Description":"Citrus (except Orange) Groves"},{"NAICSCode":111331,"Description":"Apple Orchards"},{"NAICSCode":111332,"Description":"Grape Vineyards"},{"NAICSCode":111333,"Description":"Strawberry Farming"},{"NAICSCode":111334,"Description":"Berry (except Strawberry) Farming"},{"NAICSCode":111335,"Description":"Tree Nut Farming"},{"NAICSCode":111336,"Description":"Fruit and Tree Nut Combination Farming"},{"NAICSCode":111339,"Description":"Other Noncitrus Fruit Farming"},{"NAICSCode":111411,"Description":"Mushroom Production"},{"NAICSCode":111419,"Description":"Other Food Crops Grown Under Cover"},{"NAICSCode":111421,"Description":"Nursery and Tree Production"},{"NAICSCode":111422,"Description":"Floriculture Production"},{"NAICSCode":111910,"Description":"Tobacco Farming"},{"NAICSCode":111920,"Description":"Cotton Farming"},{"NAICSCode":111930,"Description":"Sugarcane Farming"},{"NAICSCode":111940,"Description":"Hay Farming"},{"NAICSCode":111991,"Description":"Sugar Beet Farming"},{"NAICSCode":111992,"Description":"Peanut Farming"},{"NAICSCode":111998,"Description":"All Other Miscellaneous Crop Farming"},{"NAICSCode":112111,"Description":"Beef Cattle Ranching and Farming"},{"NAICSCode":112112,"Description":"Cattle Feedlots"},{"NAICSCode":112120,"Description":"Dairy Cattle and Milk Production"},{"NAICSCode":112130,"Description":"Dual-Purpose Cattle Ranching and Farming"},{"NAICSCode":112210,"Description":"Hog and Pig Farming"},{"NAICSCode":112310,"Description":"Chicken Egg Production"},{"NAICSCode":112320,"Description":"Broilers and Other Meat Type Chicken Production"},{"NAICSCode":112330,"Description":"Turkey Production"},{"NAICSCode":112340,"Description":"Poultry Hatcheries"},{"NAICSCode":112390,"Description":"Other Poultry Production"},{"NAICSCode":112410,"Description":"Sheep Farming"},{"NAICSCode":112420,"Description":"Goat Farming"},{"NAICSCode":112511,"Description":"Finfish Farming and Fish Hatcheries"},{"NAICSCode":112512,"Description":"Shellfish Farming"},{"NAICSCode":112519,"Description":"Other Aquaculture"},{"NAICSCode":112910,"Description":"Apiculture"},{"NAICSCode":112920,"Description":"Horses and Other Equine Production"},{"NAICSCode":112930,"Description":"Fur-Bearing Animal and Rabbit Production"},{"NAICSCode":112990,"Description":"All Other Animal Production"},{"NAICSCode":113110,"Description":"Timber Tract Operations"},{"NAICSCode":113210,"Description":"Forest Nurseries and Gathering of Forest Products"},{"NAICSCode":113310,"Description":"Logging"},{"NAICSCode":114111,"Description":"Finfish Fishing"},{"NAICSCode":114112,"Description":"Shellfish Fishing"},{"NAICSCode":114119,"Description":"Other Marine Fishing"},{"NAICSCode":114210,"Description":"Hunting and Trapping"},{"NAICSCode":115111,"Description":"Cotton Ginning"},{"NAICSCode":115112,"Description":"Soil Preparation, Planting, and Cultivating"},{"NAICSCode":115113,"Description":"Crop Harvesting, Primarily by Machine"},{"NAICSCode":115114,"Description":"Postharvest Crop Activities (except Cotton Ginning)"},{"NAICSCode":115115,"Description":"Farm Labor Contractors and Crew Leaders"},{"NAICSCode":115116,"Description":"Farm Management Services"},{"NAICSCode":115210,"Description":"Support Activities for Animal Production"},{"NAICSCode":115310,"Description":"Support Activities for Forestry"},{"NAICSCode":211120,"Description":"Crude Petroleum Extraction"},{"NAICSCode":211130,"Description":"Natural Gas Extraction"},{"NAICSCode":211130,"Description":"Natural Gas Extraction"},{"NAICSCode":212111,"Description":"Bituminous Coal and Lignite Surface Mining"},{"NAICSCode":212112,"Description":"Bituminous Coal Underground Mining"},{"NAICSCode":212113,"Description":"Anthracite Mining"},{"NAICSCode":212210,"Description":"Iron Ore Mining"},{"NAICSCode":212221,"Description":"Gold Ore Mining"},{"NAICSCode":212222,"Description":"Silver Ore Mining"},{"NAICSCode":212230,"Description":"Copper, Nickel, Lead, and Zinc Mining"},{"NAICSCode":212230,"Description":"Copper, Nickel, Lead, and Zinc Mining"},{"NAICSCode":212291,"Description":"Uranium-Radium-Vanadium Ore Mining"},{"NAICSCode":212299,"Description":"All Other Metal Ore Mining"},{"NAICSCode":212311,"Description":"Dimension Stone Mining and Quarrying"},{"NAICSCode":212312,"Description":"Crushed and Broken Limestone Mining and Quarrying"},{"NAICSCode":212313,"Description":"Crushed and Broken Granite Mining and Quarrying"},{"NAICSCode":212319,"Description":"Other Crushed and Broken Stone Mining and Quarrying"},{"NAICSCode":212321,"Description":"Construction Sand and Gravel Mining"},{"NAICSCode":212322,"Description":"Industrial Sand Mining"},{"NAICSCode":212324,"Description":"Kaolin and Ball Clay Mining"},{"NAICSCode":212325,"Description":"Clay and Ceramic and Refractory Minerals Mining"},{"NAICSCode":212391,"Description":"Potash, Soda, and Borate Mineral Mining"},{"NAICSCode":212392,"Description":"Phosphate Rock Mining"},{"NAICSCode":212393,"Description":"Other Chemical and Fertilizer Mineral Mining"},{"NAICSCode":212399,"Description":"All Other Nonmetallic Mineral Mining"},{"NAICSCode":213111,"Description":"Drilling Oil and Gas Wells"},{"NAICSCode":213112,"Description":"Support Activities for Oil and Gas Operations"},{"NAICSCode":213113,"Description":"Support Activities for Coal Mining"},{"NAICSCode":213114,"Description":"Support Activities for Metal Mining"},{"NAICSCode":213115,"Description":"Support Activities for Nonmetallic Minerals (except Fuels) Mining"},{"NAICSCode":221111,"Description":"Hydroelectric Power Generation"},{"NAICSCode":221112,"Description":"Fossil Fuel Electric Power Generation"},{"NAICSCode":221113,"Description":"Nuclear Electric Power Generation"},{"NAICSCode":221114,"Description":"Solar Electric Power Generation"},{"NAICSCode":221115,"Description":"Wind Electric Power Generation"},{"NAICSCode":221116,"Description":"Geothermal Electric Power Generation"},{"NAICSCode":221117,"Description":"Biomass Electric Power Generation"},{"NAICSCode":221118,"Description":"Other Electric Power Generation"},{"NAICSCode":221121,"Description":"Electric Bulk Power Transmission and Control"},{"NAICSCode":221122,"Description":"Electric Power Distribution"},{"NAICSCode":221210,"Description":"Natural Gas Distribution"},{"NAICSCode":221310,"Description":"Water Supply and Irrigation Systems"},{"NAICSCode":221320,"Description":"Sewage Treatment Facilities"},{"NAICSCode":221330,"Description":"Steam and Air-Conditioning Supply"},{"NAICSCode":236115,"Description":"New Single-Family Housing Construction (except For-Sale Builders)"},{"NAICSCode":236116,"Description":"New Multifamily Housing Construction (except For-Sale Builders)"},{"NAICSCode":236117,"Description":"New Housing For-Sale Builders"},{"NAICSCode":236118,"Description":"Residential Remodelers"},{"NAICSCode":236210,"Description":"Industrial Building Construction"},{"NAICSCode":236220,"Description":"Commercial and Institutional Building Construction"},{"NAICSCode":237110,"Description":"Water and Sewer Line and Related Structures Construction"},{"NAICSCode":237120,"Description":"Oil and Gas Pipeline and Related Structures Construction"},{"NAICSCode":237130,"Description":"Power and Communication Line and Related Structures Construction"},{"NAICSCode":237210,"Description":"Land Subdivision"},{"NAICSCode":237310,"Description":"Highway, Street, and Bridge Construction"},{"NAICSCode":237990,"Description":"Other Heavy and Civil Engineering Construction"},{"NAICSCode":238110,"Description":"Poured Concrete Foundation and Structure Contractors"},{"NAICSCode":238120,"Description":"Structural Steel and Precast Concrete Contractors"},{"NAICSCode":238130,"Description":"Framing Contractors"},{"NAICSCode":238140,"Description":"Masonry Contractors"},{"NAICSCode":238150,"Description":"Glass and Glazing Contractors"},{"NAICSCode":238160,"Description":"Roofing Contractors"},{"NAICSCode":238170,"Description":"Siding Contractors"},{"NAICSCode":238190,"Description":"Other Foundation, Structure, and Building Exterior Contractors"},{"NAICSCode":238210,"Description":"Electrical Contractors and Other Wiring Installation Contractors"},{"NAICSCode":238220,"Description":"Plumbing, Heating, and Air-Conditioning Contractors"},{"NAICSCode":238290,"Description":"Other Building Equipment Contractors"},{"NAICSCode":238310,"Description":"Drywall and Insulation Contractors"},{"NAICSCode":238320,"Description":"Painting and Wall Covering Contractors"},{"NAICSCode":238330,"Description":"Flooring Contractors"},{"NAICSCode":238340,"Description":"Tile and Terrazzo Contractors"},{"NAICSCode":238350,"Description":"Finish Carpentry Contractors"},{"NAICSCode":238390,"Description":"Other Building Finishing Contractors"},{"NAICSCode":238910,"Description":"Site Preparation Contractors"},{"NAICSCode":238990,"Description":"All Other Specialty Trade Contractors"},{"NAICSCode":311111,"Description":"Dog and Cat Food Manufacturing"},{"NAICSCode":311119,"Description":"Other Animal Food Manufacturing"},{"NAICSCode":311211,"Description":"Flour Milling"},{"NAICSCode":311212,"Description":"Rice Milling"},{"NAICSCode":311213,"Description":"Malt Manufacturing"},{"NAICSCode":311221,"Description":"Wet Corn Milling"},{"NAICSCode":311224,"Description":"Soybean and Other Oilseed Processing"},{"NAICSCode":311225,"Description":"Fats and Oils Refining and Blending"},{"NAICSCode":311230,"Description":"Breakfast Cereal Manufacturing"},{"NAICSCode":311313,"Description":"Beet Sugar Manufacturing"},{"NAICSCode":311314,"Description":"Cane Sugar Manufacturing"},{"NAICSCode":311340,"Description":"Nonchocolate Confectionery Manufacturing"},{"NAICSCode":311351,"Description":"Chocolate and Confectionery Manufacturing from Cacao Beans"},{"NAICSCode":311352,"Description":"Confectionery Manufacturing from Purchased Chocolate"},{"NAICSCode":311411,"Description":"Frozen Fruit, Juice, and Vegetable Manufacturing"},{"NAICSCode":311412,"Description":"Frozen Specialty Food Manufacturing"},{"NAICSCode":311421,"Description":"Fruit and Vegetable Canning"},{"NAICSCode":311422,"Description":"Specialty Canning"},{"NAICSCode":311423,"Description":"Dried and Dehydrated Food Manufacturing"},{"NAICSCode":311511,"Description":"Fluid Milk Manufacturing"},{"NAICSCode":311512,"Description":"Creamery Butter Manufacturing"},{"NAICSCode":311513,"Description":"Cheese Manufacturing"},{"NAICSCode":311514,"Description":"Dry, Condensed, and Evaporated Dairy Product Manufacturing"},{"NAICSCode":311520,"Description":"Ice Cream and Frozen Dessert Manufacturing"},{"NAICSCode":311611,"Description":"Animal (except Poultry) Slaughtering"},{"NAICSCode":311612,"Description":"Meat Processed from Carcasses"},{"NAICSCode":311613,"Description":"Rendering and Meat Byproduct Processing"},{"NAICSCode":311615,"Description":"Poultry Processing"},{"NAICSCode":311710,"Description":"Seafood Product Preparation and Packaging"},{"NAICSCode":311811,"Description":"Retail Bakeries"},{"NAICSCode":311812,"Description":"Commercial Bakeries"},{"NAICSCode":311813,"Description":"Frozen Cakes, Pies, and Other Pastries Manufacturing"},{"NAICSCode":311821,"Description":"Cookie and Cracker Manufacturing"},{"NAICSCode":311824,"Description":"Dry Pasta, Dough, and Flour Mixes Manufacturing from Purchased Flour"},{"NAICSCode":311830,"Description":"Tortilla Manufacturing"},{"NAICSCode":311911,"Description":"Roasted Nuts and Peanut Butter Manufacturing"},{"NAICSCode":311919,"Description":"Other Snack Food Manufacturing"},{"NAICSCode":311920,"Description":"Coffee and Tea Manufacturing"},{"NAICSCode":311930,"Description":"Flavoring Syrup and Concentrate Manufacturing"},{"NAICSCode":311941,"Description":"Mayonnaise, Dressing, and Other Prepared Sauce Manufacturing"},{"NAICSCode":311942,"Description":"Spice and Extract Manufacturing"},{"NAICSCode":311991,"Description":"Perishable Prepared Food Manufacturing"},{"NAICSCode":311999,"Description":"All Other Miscellaneous Food Manufacturing"},{"NAICSCode":312111,"Description":"Soft Drink Manufacturing"},{"NAICSCode":312112,"Description":"Bottled Water Manufacturing"},{"NAICSCode":312113,"Description":"Ice Manufacturing"},{"NAICSCode":312120,"Description":"Breweries"},{"NAICSCode":312130,"Description":"Wineries"},{"NAICSCode":312140,"Description":"Distilleries"},{"NAICSCode":312230,"Description":"Tobacco Manufacturing"},{"NAICSCode":313110,"Description":"Fiber, Yarn, and Thread Mills"},{"NAICSCode":313210,"Description":"Broadwoven Fabric Mills"},{"NAICSCode":313220,"Description":"Narrow Fabric Mills and Schiffli Machine Embroidery"},{"NAICSCode":313230,"Description":"Nonwoven Fabric Mills"},{"NAICSCode":313240,"Description":"Knit Fabric Mills"},{"NAICSCode":313310,"Description":"Textile and Fabric Finishing Mills"},{"NAICSCode":313320,"Description":"Fabric Coating Mills"},{"NAICSCode":314110,"Description":"Carpet and Rug Mills"},{"NAICSCode":314120,"Description":"Curtain and Linen Mills"},{"NAICSCode":314910,"Description":"Textile Bag and Canvas Mills"},{"NAICSCode":314994,"Description":"Rope, Cordage, Twine, Tire Cord, and Tire Fabric Mills"},{"NAICSCode":314999,"Description":"All Other Miscellaneous Textile Product Mills"},{"NAICSCode":315110,"Description":"Hosiery and Sock Mills"},{"NAICSCode":315190,"Description":"Other Apparel Knitting Mills"},{"NAICSCode":315210,"Description":"Cut and Sew Apparel Contractors"},{"NAICSCode":315220,"Description":"Men's and Boys' Cut and Sew Apparel Manufacturing"},{"NAICSCode":315240,"Description":"Women's, Girls', and Infants' Cut and Sew Apparel Manufacturing"},{"NAICSCode":315280,"Description":"Other Cut and Sew Apparel Manufacturing"},{"NAICSCode":315990,"Description":"Apparel Accessories and Other Apparel Manufacturing"},{"NAICSCode":316110,"Description":"Leather and Hide Tanning and Finishing"},{"NAICSCode":316210,"Description":"Footwear Manufacturing"},{"NAICSCode":316992,"Description":"Women's Handbag and Purse Manufacturing"},{"NAICSCode":316998,"Description":"All Other Leather Good and Allied Product Manufacturing"},{"NAICSCode":321113,"Description":"Sawmills"},{"NAICSCode":321114,"Description":"Wood Preservation"},{"NAICSCode":321211,"Description":"Hardwood Veneer and Plywood Manufacturing"},{"NAICSCode":321212,"Description":"Softwood Veneer and Plywood Manufacturing"},{"NAICSCode":321213,"Description":"Engineered Wood Member (except Truss) Manufacturing"},{"NAICSCode":321214,"Description":"Truss Manufacturing"},{"NAICSCode":321219,"Description":"Reconstituted Wood Product Manufacturing"},{"NAICSCode":321911,"Description":"Wood Window and Door Manufacturing"},{"NAICSCode":321912,"Description":"Cut Stock, Resawing Lumber, and Planing"},{"NAICSCode":321918,"Description":"Other Millwork (including Flooring)"},{"NAICSCode":321920,"Description":"Wood Container and Pallet Manufacturing"},{"NAICSCode":321991,"Description":"Manufactured Home (Mobile Home) Manufacturing"},{"NAICSCode":321992,"Description":"Prefabricated Wood Building Manufacturing"},{"NAICSCode":321999,"Description":"All Other Miscellaneous Wood Product Manufacturing"},{"NAICSCode":322110,"Description":"Pulp Mills"},{"NAICSCode":322121,"Description":"Paper (except Newsprint) Mills"},{"NAICSCode":322122,"Description":"Newsprint Mills"},{"NAICSCode":322130,"Description":"Paperboard Mills"},{"NAICSCode":322211,"Description":"Corrugated and Solid Fiber Box Manufacturing"},{"NAICSCode":322212,"Description":"Folding Paperboard Box Manufacturing"},{"NAICSCode":322219,"Description":"Other Paperboard Container Manufacturing"},{"NAICSCode":322220,"Description":"Paper Bag and Coated and Treated Paper Manufacturing"},{"NAICSCode":322230,"Description":"Stationery Product Manufacturing"},{"NAICSCode":322291,"Description":"Sanitary Paper Product Manufacturing"},{"NAICSCode":322299,"Description":"All Other Converted Paper Product Manufacturing"},{"NAICSCode":323111,"Description":"Commercial Printing (except Screen and Books)"},{"NAICSCode":323113,"Description":"Commercial Screen Printing"},{"NAICSCode":323117,"Description":"Books Printing"},{"NAICSCode":323120,"Description":"Support Activites for Printing"},{"NAICSCode":324110,"Description":"Petroleum Refineries"},{"NAICSCode":324121,"Description":"Asphalt Paving Mixture and Block Manufacturing"},{"NAICSCode":324122,"Description":"Asphalt Shingle and Coating Materials Manufacturing"},{"NAICSCode":324191,"Description":"Petroleum Lubricating Oil and Grease Manufacturing"},{"NAICSCode":324199,"Description":"All Other Petroleum and Coal Products Manufacturing"},{"NAICSCode":325110,"Description":"Petrochemical Manufacturing"},{"NAICSCode":325120,"Description":"Industrial Gas Manufacturing"},{"NAICSCode":325130,"Description":"Synthetic Dye and Pigment Manufacturing"},{"NAICSCode":325180,"Description":"Other Basic Inorganic Chemical Manufacturing"},{"NAICSCode":325193,"Description":"Ethyl Alcohol Manufacturing"},{"NAICSCode":325194,"Description":"Cyclic Crude, Intermediate, and Gum and Wood Chemical Manufacturing"},{"NAICSCode":325199,"Description":"All Other Basic Organic Chemical Manufacturing"},{"NAICSCode":325211,"Description":"Plastics Material and Resin Manufacturing"},{"NAICSCode":325212,"Description":"Synthetic Rubber Manufacturing"},{"NAICSCode":325220,"Description":"Artificial and Synthetic Fibers and Filaments Manufacturing"},{"NAICSCode":325311,"Description":"Nitrogenous Fertilizer Manufacturing"},{"NAICSCode":325312,"Description":"Phosphatic Fertilizer Manufacturing"},{"NAICSCode":325314,"Description":"Fertilizer (Mixing Only) Manufacturing"},{"NAICSCode":325320,"Description":"Pesticide and Other Agricultural Chemical Manufacturing"},{"NAICSCode":325411,"Description":"Medicinal and Botanical Manufacturing"},{"NAICSCode":325412,"Description":"Pharmaceutical Preparation Manufacturing"},{"NAICSCode":325413,"Description":"In-Vitro Diagnostic Substance Manufacturing"},{"NAICSCode":325414,"Description":"Biological Product (except Diagnostic) Manufacturing"},{"NAICSCode":325510,"Description":"Paint and Coating Manufacturing"},{"NAICSCode":325520,"Description":"Adhesive Manufacturing"},{"NAICSCode":325611,"Description":"Soap and Other Detergent Manufacturing"},{"NAICSCode":325612,"Description":"Polish and Other Sanitation Good Manufacturing"},{"NAICSCode":325613,"Description":"Surface Active Agent Manufacturing"},{"NAICSCode":325620,"Description":"Toilet Preparation Manufacturing"},{"NAICSCode":325910,"Description":"Printing Ink Manufacturing"},{"NAICSCode":325920,"Description":"Explosives Manufacturing"},{"NAICSCode":325991,"Description":"Custom Compounding of Purchased Resins"},{"NAICSCode":325992,"Description":"Photographic Film, Paper, Plate, and Chemical Manufacturing"},{"NAICSCode":325998,"Description":"All Other Miscellaneous Chemical Product and Preparation Manufacturing"},{"NAICSCode":326111,"Description":"Plastics Bag and Pouch Manufacturing"},{"NAICSCode":326112,"Description":"Plastics Packaging Film and Sheet (including Laminated) Manufacturing"},{"NAICSCode":326113,"Description":"Unlaminated Plastics Film and Sheet (except Packaging) Manufacturing"},{"NAICSCode":326121,"Description":"Unlaminated Plastics Profile Shape Manufacturing"},{"NAICSCode":326122,"Description":"Plastics Pipe and Pipe Fitting Manufacturing"},{"NAICSCode":326130,"Description":"Laminated Plastics Plate, Sheet (except Packaging), and Shape Manufacturing"},{"NAICSCode":326140,"Description":"Polystyrene Foam Product Manufacturing"},{"NAICSCode":326150,"Description":"Urethane and Other Foam Product (except Polystyrene) Manufacturing"},{"NAICSCode":326160,"Description":"Plastics Bottle Manufacturing"},{"NAICSCode":326191,"Description":"Plastics Plumbing Fixture Manufacturing"},{"NAICSCode":326199,"Description":"All Other Plastics Product Manufacturing"},{"NAICSCode":326211,"Description":"Tire Manufacturing (except Retreading)"},{"NAICSCode":326212,"Description":"Tire Retreading"},{"NAICSCode":326220,"Description":"Rubber and Plastics Hoses and Belting Manufacturing"},{"NAICSCode":326291,"Description":"Rubber Product Manufacturing for Mechanical Use"},{"NAICSCode":326299,"Description":"All Other Rubber Product Manufacturing"},{"NAICSCode":327110,"Description":"Pottery, Ceramics, and Plumbing Fixture Manufacturing"},{"NAICSCode":327120,"Description":"Clay Building Material and Refractories Manufacturing"},{"NAICSCode":327211,"Description":"Flat Glass Manufacturing"},{"NAICSCode":327212,"Description":"Other Pressed and Blown Glass and Glassware Manufacturing"},{"NAICSCode":327213,"Description":"Glass Container Manufacturing"},{"NAICSCode":327215,"Description":"Glass Product Manufacturing Made of Purchased Glass"},{"NAICSCode":327310,"Description":"Cement Manufacturing"},{"NAICSCode":327320,"Description":"Ready-Mix Concrete Manufacturing"},{"NAICSCode":327331,"Description":"Concrete Block and Brick Manufacturing"},{"NAICSCode":327332,"Description":"Concrete Pipe Manufacturing"},{"NAICSCode":327390,"Description":"Other Concrete Product Manufacturing"},{"NAICSCode":327410,"Description":"Lime Manufacturing"},{"NAICSCode":327420,"Description":"Gypsum Product Manufacturing"},{"NAICSCode":327910,"Description":"Abrasive Product Manufacturing"},{"NAICSCode":327991,"Description":"Cut Stone and Stone Product Manufacturing"},{"NAICSCode":327992,"Description":"Ground or Treated Mineral and Earth Manufacturing"},{"NAICSCode":327993,"Description":"Mineral Wool Manufacturing"},{"NAICSCode":327999,"Description":"All Other Miscellaneous Nonmetallic Mineral Product Manufacturing"},{"NAICSCode":331110,"Description":"Iron and Steel Mills and Ferroalloy Manufacturing"},{"NAICSCode":331210,"Description":"Iron and Steel Pipe and Tube Manufacturing from Purchased Steel"},{"NAICSCode":331221,"Description":"Rolled Steel Shape Manufacturing"},{"NAICSCode":331222,"Description":"Steel Wire Drawing"},{"NAICSCode":331313,"Description":"Alumina Refining and Primary Aluminum Production"},{"NAICSCode":331314,"Description":"Secondary Smelting and Alloying of Aluminum"},{"NAICSCode":331315,"Description":"Aluminum Sheet, Plate, and Foil Manufacturing"},{"NAICSCode":331318,"Description":"Other Aluminum Rolling, Drawing, and Extruding"},{"NAICSCode":331410,"Description":"Nonferrous Metal (except Aluminum) Smelting and Refining"},{"NAICSCode":331420,"Description":"Copper Rolling, Drawing, Extruding, and Alloying"},{"NAICSCode":331491,"Description":"Nonferrous Metal (except Copper and Aluminum) Rolling, Drawing, and Extruding"},{"NAICSCode":331492,"Description":"Secondary Smelting, Refining, and Alloying of Nonferrous Metal (except Copper and Aluminum)"},{"NAICSCode":331511,"Description":"Iron Foundries"},{"NAICSCode":331512,"Description":"Steel Investment Foundries"},{"NAICSCode":331513,"Description":"Steel Foundries (except Investment)"},{"NAICSCode":331523,"Description":"Nonferrous Metal Die-Casting Foundries"},{"NAICSCode":331524,"Description":"Aluminum Foundries (except Die-Casting)"},{"NAICSCode":331529,"Description":"Other Nonferrous Metal Foundries (except Die-Casting)"},{"NAICSCode":332111,"Description":"Iron and Steel Forging"},{"NAICSCode":332112,"Description":"Nonferrous Forging"},{"NAICSCode":332114,"Description":"Custom Roll Forming"},{"NAICSCode":332117,"Description":"Powder Metallurgy Part Manufacturing"},{"NAICSCode":332119,"Description":"Metal Crown, Closure, and Other Metal Stamping (except Automotive)"},{"NAICSCode":332215,"Description":"Metal Kitchen Cookware, Utensil, Cutlery, and Flatware (except Precious) Manufacturing"},{"NAICSCode":332216,"Description":"Saw Blade and Handtool Manufacturing"},{"NAICSCode":332311,"Description":"Prefabricated Metal Building and Component Manufacturing"},{"NAICSCode":332312,"Description":"Fabricated Structural Metal Manufacturing"},{"NAICSCode":332313,"Description":"Plate Work Manufacturing"},{"NAICSCode":332321,"Description":"Metal Window and Door Manufacturing"},{"NAICSCode":332322,"Description":"Sheet Metal Work Manufacturing"},{"NAICSCode":332323,"Description":"Ornamental and Architectural Metal Work Manufacturing"},{"NAICSCode":332410,"Description":"Power Boiler and Heat Exchanger Manufacturing"},{"NAICSCode":332420,"Description":"Metal Tank (Heavy Gauge) Manufacturing"},{"NAICSCode":332431,"Description":"Metal Can Manufacturing"},{"NAICSCode":332439,"Description":"Other Metal Container Manufacturing"},{"NAICSCode":332510,"Description":"Hardware Manufacturing"},{"NAICSCode":332613,"Description":"Spring Manufacturing"},{"NAICSCode":332618,"Description":"Other Fabricated Wire Product Manufacturing"},{"NAICSCode":332710,"Description":"Machine Shops"},{"NAICSCode":332721,"Description":"Precision Turned Product Manufacturing"},{"NAICSCode":332722,"Description":"Bolt, Nut, Screw, Rivet, and Washer Manufacturing"},{"NAICSCode":332811,"Description":"Metal Heat Treating"},{"NAICSCode":332812,"Description":"Metal Coating, Engraving (except Jewelry and Silverware), and Allied Services to Manufacturers"},{"NAICSCode":332813,"Description":"Electroplating, Plating, Polishing, Anodizing, and Coloring"},{"NAICSCode":332911,"Description":"Industrial Valve Manufacturing"},{"NAICSCode":332912,"Description":"Fluid Power Valve and Hose Fitting Manufacturing"},{"NAICSCode":332913,"Description":"Plumbing Fixture Fitting and Trim Manufacturing"},{"NAICSCode":332919,"Description":"Other Metal Valve and Pipe Fitting Manufacturing"},{"NAICSCode":332991,"Description":"Ball and Roller Bearing Manufacturing"},{"NAICSCode":332992,"Description":"Small Arms Ammunition Manufacturing"},{"NAICSCode":332993,"Description":"Ammunition (except Small Arms) Manufacturing"},{"NAICSCode":332994,"Description":"Small Arms, Ordnance, and Ordnance Accessories Manufacturing"},{"NAICSCode":332996,"Description":"Fabricated Pipe and Pipe Fitting Manufacturing"},{"NAICSCode":332999,"Description":"All Other Miscellaneous Fabricated Metal Product Manufacturing"},{"NAICSCode":333111,"Description":"Farm Machinery and Equipment Manufacturing"},{"NAICSCode":333112,"Description":"Lawn and Garden Tractor and Home Lawn and Garden Equipment Manufacturing"},{"NAICSCode":333120,"Description":"Construction Machinery Manufacturing"},{"NAICSCode":333131,"Description":"Mining Machinery and Equipment Manufacturing"},{"NAICSCode":333132,"Description":"Oil and Gas Field Machinery and Equipment Manufacturing"},{"NAICSCode":333241,"Description":"Food Product Machinery Manufacturing"},{"NAICSCode":333242,"Description":"Semiconductor Machinery Manufacturing"},{"NAICSCode":333243,"Description":"Sawmill, Woodworking, and Paper Machinery Manufacturing"},{"NAICSCode":333244,"Description":"Printing Machinery and Equipment Manufacturing"},{"NAICSCode":333249,"Description":"Other Industrial Machinery Manufacturing"},{"NAICSCode":333314,"Description":"Optical Instrument and Lens Manufacturing"},{"NAICSCode":333316,"Description":"Photographic and Photocopying Equipment Manufacturing"},{"NAICSCode":333318,"Description":"Other Commercial and Service Industry Machinery Manufacturing"},{"NAICSCode":333413,"Description":"Industrial and Commercial Fan and Blower and Air Purification Equipment Manufacturing"},{"NAICSCode":333414,"Description":"Heating Equipment (except Warm Air Furnaces) Manufacturing"},{"NAICSCode":333415,"Description":"Air-Conditioning and Warm Air Heating Equipment and Commercial and Industrial Refrigeration Equipment Manufacturing"},{"NAICSCode":333511,"Description":"Industrial Mold Manufacturing"},{"NAICSCode":333514,"Description":"Special Die and Tool, Die Set, Jig, and Fixture Manufacturing"},{"NAICSCode":333515,"Description":"Cutting Tool and Machine Tool Accessory Manufacturing"},{"NAICSCode":333517,"Description":"Machine Tool Manufacturing"},{"NAICSCode":333519,"Description":"Rolling Mill and Other Metalworking Machinery Manufacturing"},{"NAICSCode":333611,"Description":"Turbine and Turbine Generator Set Units Manufacturing"},{"NAICSCode":333612,"Description":"Speed Changer, Industrial High-Speed Drive, and Gear Manufacturing"},{"NAICSCode":333613,"Description":"Mechanical Power Transmission Equipment Manufacturing"},{"NAICSCode":333618,"Description":"Other Engine Equipment Manufacturing"},{"NAICSCode":333912,"Description":"Air and Gas Compressor Manufacturing"},{"NAICSCode":333914,"Description":"Measuring, Dispensing, and Other Pumping Equipment Manufacturing"},{"NAICSCode":333914,"Description":"Measuring, Dispensing, and Other Pumping Equipment Manufacturing"},{"NAICSCode":333921,"Description":"Elevator and Moving Stairway Manufacturing"},{"NAICSCode":333922,"Description":"Conveyor and Conveying Equipment Manufacturing"},{"NAICSCode":333923,"Description":"Overhead Traveling Crane, Hoist, and Monorail System Manufacturing"},{"NAICSCode":333924,"Description":"Industrial Truck, Tractor, Trailer, and Stacker Machinery Manufacturing"},{"NAICSCode":333991,"Description":"Power-Driven Handtool Manufacturing"},{"NAICSCode":333992,"Description":"Welding and Soldering Equipment Manufacturing"},{"NAICSCode":333993,"Description":"Packaging Machinery Manufacturing"},{"NAICSCode":333994,"Description":"Industrial Process Furnace and Oven Manufacturing"},{"NAICSCode":333995,"Description":"Fluid Power Cylinder and Actuator Manufacturing"},{"NAICSCode":333996,"Description":"Fluid Power Pump and Motor Manufacturing"},{"NAICSCode":333997,"Description":"Scale and Balance Manufacturing"},{"NAICSCode":333999,"Description":"All Other Miscellaneous General Purpose Machinery Manufacturing"},{"NAICSCode":334111,"Description":"Electronic Computer Manufacturing"},{"NAICSCode":334112,"Description":"Computer Storage Device Manufacturing"},{"NAICSCode":334118,"Description":"Computer Terminal and Other Computer Peripheral Equipment Manufacturing"},{"NAICSCode":334210,"Description":"Telephone Apparatus Manufacturing"},{"NAICSCode":334220,"Description":"Radio and Television Broadcasting and Wireless Communications Equipment Manufacturing"},{"NAICSCode":334290,"Description":"Other Communications Equipment Manufacturing"},{"NAICSCode":334310,"Description":"Audio and Video Equipment Manufacturing"},{"NAICSCode":334412,"Description":"Bare Printed Circuit Board Manufacturing"},{"NAICSCode":334413,"Description":"Semiconductor and Related Device Manufacturing"},{"NAICSCode":334416,"Description":"Capacitor, Resistor, Coil, Transformer, and Other Inductor Manufacturing"},{"NAICSCode":334417,"Description":"Electronic Connector Manufacturing"},{"NAICSCode":334418,"Description":"Printed Circuit Assembly (Electronic Assembly) Manufacturing"},{"NAICSCode":334419,"Description":"Other Electronic Component Manufacturing"},{"NAICSCode":334510,"Description":"Electromedical and Electrotherapeutic Apparatus Manufacturing"},{"NAICSCode":334511,"Description":"Search, Detection, Navigation, Guidance, Aeronautical, and Nautical System and Instrument Manufacturing"},{"NAICSCode":334512,"Description":"Automatic Environmental Control Manufacturing for Residential, Commercial, and Appliance Use"},{"NAICSCode":334513,"Description":"Instruments and Related Products Manufacturing for Measuring, Displaying, and Controlling Industrial Process Variables"},{"NAICSCode":334514,"Description":"Totalizing Fluid Meter and Counting Device Manufacturing"},{"NAICSCode":334515,"Description":"Instrument Manufacturing for Measuring and Testing Electricity and Electrical Signals"},{"NAICSCode":334516,"Description":"Analytical Laboratory Instrument Manufacturing"},{"NAICSCode":334517,"Description":"Irradiation Apparatus Manufacturing"},{"NAICSCode":334519,"Description":"Other Measuring and Controlling Device Manufacturing"},{"NAICSCode":334613,"Description":"Blank Magnetic and Optical Recording Media Manufacturing"},{"NAICSCode":334614,"Description":"Software and Other Prerecorded Compact Disc, Tape, and Record Reproducing"},{"NAICSCode":335110,"Description":"Electric Lamp Bulb and Part Manufacturing"},{"NAICSCode":335121,"Description":"Residential Electric Lighting Fixture Manufacturing"},{"NAICSCode":335122,"Description":"Commercial, Industrial, and Institutional Electric Lighting Fixture Manufacturing"},{"NAICSCode":335129,"Description":"Other Lighting Equipment Manufacturing"},{"NAICSCode":335210,"Description":"Small Electrical Appliance Manufacturing"},{"NAICSCode":335220,"Description":"Major Household Appliance Manufacturing"},{"NAICSCode":335220,"Description":"Major Household Appliance Manufacturing"},{"NAICSCode":335220,"Description":"Major Household Appliance Manufacturing"},{"NAICSCode":335220,"Description":"Major Household Appliance Manufacturing"},{"NAICSCode":335311,"Description":"Power, Distribution, and Specialty Transformer Manufacturing"},{"NAICSCode":335312,"Description":"Motor and Generator Manufacturing"},{"NAICSCode":335313,"Description":"Switchgear and Switchboard Apparatus Manufacturing"},{"NAICSCode":335314,"Description":"Relay and Industrial Control Manufacturing"},{"NAICSCode":335911,"Description":"Storage Battery Manufacturing"},{"NAICSCode":335912,"Description":"Primary Battery Manufacturing"},{"NAICSCode":335921,"Description":"Fiber Optic Cable Manufacturing"},{"NAICSCode":335929,"Description":"Other Communication and Energy Wire Manufacturing"},{"NAICSCode":335931,"Description":"Current-Carrying Wiring Device Manufacturing"},{"NAICSCode":335932,"Description":"Noncurrent-Carrying Wiring Device Manufacturing"},{"NAICSCode":335991,"Description":"Carbon and Graphite Product Manufacturing"},{"NAICSCode":335999,"Description":"All Other Miscellaneous Electrical Equipment and Component Manufacturing"},{"NAICSCode":336111,"Description":"Automobile Manufacturing"},{"NAICSCode":336112,"Description":"Light Truck and Utility Vehicle Manufacturing"},{"NAICSCode":336120,"Description":"Heavy Duty Truck Manufacturing"},{"NAICSCode":336211,"Description":"Motor Vehicle Body Manufacturing"},{"NAICSCode":336212,"Description":"Truck Trailer Manufacturing"},{"NAICSCode":336213,"Description":"Motor Home Manufacturing"},{"NAICSCode":336214,"Description":"Travel Trailer and Camper Manufacturing"},{"NAICSCode":336310,"Description":"Motor Vehicle Gasoline Engine and Engine Parts Manufacturing"},{"NAICSCode":336320,"Description":"Motor Vehicle Electrical and Electronic Equipment Manufacturing"},{"NAICSCode":336330,"Description":"Motor Vehicle Steering and Suspension Components (except Spring) Manufacturing"},{"NAICSCode":336340,"Description":"Motor Vehicle Brake System Manufacturing"},{"NAICSCode":336350,"Description":"Motor Vehicle Transmission and Power Train Parts Manufacturing"},{"NAICSCode":336360,"Description":"Motor Vehicle Seating and Interior Trim Manufacturing"},{"NAICSCode":336370,"Description":"Motor Vehicle Metal Stamping"},{"NAICSCode":336390,"Description":"Other Motor Vehicle Parts Manufacturing"},{"NAICSCode":336411,"Description":"Aircraft Manufacturing"},{"NAICSCode":336412,"Description":"Aircraft Engine and Engine Parts Manufacturing"},{"NAICSCode":336413,"Description":"Other Aircraft Parts and Auxiliary Equipment Manufacturing"},{"NAICSCode":336414,"Description":"Guided Missile and Space Vehicle Manufacturing"},{"NAICSCode":336415,"Description":"Guided Missile and Space Vehicle Propulsion Unit and Propulsion Unit Parts Manufacturing"},{"NAICSCode":336419,"Description":"Other Guided Missile and Space Vehicle Parts and Auxiliary Equipment Manufacturing"},{"NAICSCode":336510,"Description":"Railroad Rolling Stock Manufacturing"},{"NAICSCode":336611,"Description":"Ship Building and Repairing"},{"NAICSCode":336612,"Description":"Boat Building"},{"NAICSCode":336991,"Description":"Motorcycle, Bicycle, and Parts Manufacturing"},{"NAICSCode":336992,"Description":"Military Armored Vehicle, Tank, and Tank Component Manufacturing"},{"NAICSCode":336999,"Description":"All Other Transportation Equipment Manufacturing"},{"NAICSCode":337110,"Description":"Wood Kitchen Cabinet and Countertop Manufacturing"},{"NAICSCode":337121,"Description":"Upholstered Household Furniture Manufacturing"},{"NAICSCode":337122,"Description":"Nonupholstered Wood Household Furniture Manufacturing"},{"NAICSCode":337124,"Description":"Metal Household Furniture Manufacturing"},{"NAICSCode":337125,"Description":"Household Furniture (except Wood and Metal) Manufacturing"},{"NAICSCode":337127,"Description":"Institutional Furniture Manufacturing"},{"NAICSCode":337211,"Description":"Wood Office Furniture Manufacturing"},{"NAICSCode":337212,"Description":"Custom Architectural Woodwork and Millwork Manufacturing"},{"NAICSCode":337214,"Description":"Office Furniture (except Wood) Manufacturing"},{"NAICSCode":337215,"Description":"Showcase, Partition, Shelving, and Locker Manufacturing"},{"NAICSCode":337910,"Description":"Mattress Manufacturing"},{"NAICSCode":337920,"Description":"Blind and Shade Manufacturing"},{"NAICSCode":339112,"Description":"Surgical and Medical Instrument Manufacturing"},{"NAICSCode":339113,"Description":"Surgical Appliance and Supplies Manufacturing"},{"NAICSCode":339114,"Description":"Dental Equipment and Supplies Manufacturing"},{"NAICSCode":339115,"Description":"Ophthalmic Goods Manufacturing"},{"NAICSCode":339116,"Description":"Dental Laboratories"},{"NAICSCode":339910,"Description":"Jewelry and Silverware Manufacturing"},{"NAICSCode":339920,"Description":"Sporting and Athletic Goods Manufacturing"},{"NAICSCode":339930,"Description":"Doll, Toy, and Game Manufacturing"},{"NAICSCode":339940,"Description":"Office Supplies (except Paper) Manufacturing"},{"NAICSCode":339950,"Description":"Sign Manufacturing"},{"NAICSCode":339991,"Description":"Gasket, Packing, and Sealing Device Manufacturing"},{"NAICSCode":339992,"Description":"Musical Instrument Manufacturing"},{"NAICSCode":339993,"Description":"Fastener, Button, Needle, and Pin Manufacturing"},{"NAICSCode":339994,"Description":"Broom, Brush, and Mop Manufacturing"},{"NAICSCode":339995,"Description":"Burial Casket Manufacturing"},{"NAICSCode":339999,"Description":"All Other Miscellaneous Manufacturing"},{"NAICSCode":423110,"Description":"Automobile and Other Motor Vehicle Merchant Wholesalers"},{"NAICSCode":423120,"Description":"Motor Vehicle Supplies and New Parts Merchant Wholesalers"},{"NAICSCode":423130,"Description":"Tire and Tube Merchant Wholesalers"},{"NAICSCode":423140,"Description":"Motor Vehicle Parts (Used) Merchant Wholesalers"},{"NAICSCode":423210,"Description":"Furniture Merchant Wholesalers"},{"NAICSCode":423220,"Description":"Home Furnishing Merchant Wholesalers"},{"NAICSCode":423310,"Description":"Lumber, Plywood, Millwork, and Wood Panel Merchant Wholesalers"},{"NAICSCode":423320,"Description":"Brick, Stone, and Related Construction Material Merchant Wholesalers"},{"NAICSCode":423330,"Description":"Roofing, Siding, and Insulation Material Merchant Wholesalers"},{"NAICSCode":423390,"Description":"Other Construction Material Merchant Wholesalers"},{"NAICSCode":423410,"Description":"Photographic Equipment and Supplies Merchant Wholesalers"},{"NAICSCode":423420,"Description":"Office Equipment Merchant Wholesalers"},{"NAICSCode":423430,"Description":"Computer and Computer Peripheral Equipment and Software Merchant Wholesalers"},{"NAICSCode":423440,"Description":"Other Commercial Equipment Merchant Wholesalers"},{"NAICSCode":423450,"Description":"Medical, Dental, and Hospital Equipment and Supplies Merchant Wholesalers"},{"NAICSCode":423460,"Description":"Ophthalmic Goods Merchant Wholesalers"},{"NAICSCode":423490,"Description":"Other Professional Equipment and Supplies Merchant Wholesalers"},{"NAICSCode":423510,"Description":"Metal Service Centers and Other Metal Merchant Wholesalers"},{"NAICSCode":423520,"Description":"Coal and Other Mineral and Ore Merchant Wholesalers"},{"NAICSCode":423610,"Description":"Electrical Apparatus and Equipment, Wiring Supplies, and Related Equipment Merchant Wholesalers"},{"NAICSCode":423620,"Description":"Household Appliances, Electric Housewares, and Consumer Electronics Merchant Wholesalers"},{"NAICSCode":423690,"Description":"Other Electronic Parts and Equipment Merchant Wholesalers"},{"NAICSCode":423710,"Description":"Hardware Merchant Wholesalers"},{"NAICSCode":423720,"Description":"Plumbing and Heating Equipment and Supplies (Hydronics) Merchant Wholesalers"},{"NAICSCode":423730,"Description":"Warm Air Heating and Air-Conditioning Equipment and Supplies Merchant Wholesalers"},{"NAICSCode":423740,"Description":"Refrigeration Equipment and Supplies Merchant Wholesalers"},{"NAICSCode":423810,"Description":"Construction and Mining (except Oil Well) Machinery and Equipment Merchant Wholesalers"},{"NAICSCode":423820,"Description":"Farm and Garden Machinery and Equipment Merchant Wholesalers"},{"NAICSCode":423830,"Description":"Industrial Machinery and Equipment Merchant Wholesalers"},{"NAICSCode":423840,"Description":"Industrial Supplies Merchant Wholesalers"},{"NAICSCode":423850,"Description":"Service Establishment Equipment and Supplies Merchant Wholesalers"},{"NAICSCode":423860,"Description":"Transportation Equipment and Supplies (except Motor Vehicle) Merchant Wholesalers"},{"NAICSCode":423910,"Description":"Sporting and Recreational Goods and Supplies Merchant Wholesalers"},{"NAICSCode":423920,"Description":"Toy and Hobby Goods and Supplies Merchant Wholesalers"},{"NAICSCode":423930,"Description":"Recyclable Material Merchant Wholesalers"},{"NAICSCode":423940,"Description":"Jewelry, Watch, Precious Stone, and Precious Metal Merchant Wholesalers"},{"NAICSCode":423990,"Description":"Other Miscellaneous Durable Goods Merchant Wholesalers"},{"NAICSCode":424110,"Description":"Printing and Writing Paper Merchant Wholesalers"},{"NAICSCode":424120,"Description":"Stationery and Office Supplies Merchant Wholesalers"},{"NAICSCode":424130,"Description":"Industrial and Personal Service Paper Merchant Wholesalers"},{"NAICSCode":424210,"Description":"Drugs and Druggists' Sundries Merchant Wholesalers"},{"NAICSCode":424310,"Description":"Piece Goods, Notions, and Other Dry Goods Merchant Wholesalers"},{"NAICSCode":424320,"Description":"Men's and Boys' Clothing and Furnishings Merchant Wholesalers"},{"NAICSCode":424330,"Description":"Women's, Children's, and Infants' Clothing and Accessories Merchant Wholesalers"},{"NAICSCode":424340,"Description":"Footwear Merchant Wholesalers"},{"NAICSCode":424410,"Description":"General Line Grocery Merchant Wholesalers"},{"NAICSCode":424420,"Description":"Packaged Frozen Food Merchant Wholesalers"},{"NAICSCode":424430,"Description":"Dairy Product (except Dried or Canned) Merchant Wholesalers"},{"NAICSCode":424440,"Description":"Poultry and Poultry Product Merchant Wholesalers"},{"NAICSCode":424450,"Description":"Confectionery Merchant Wholesalers"},{"NAICSCode":424460,"Description":"Fish and Seafood Merchant Wholesalers"},{"NAICSCode":424470,"Description":"Meat and Meat Product Merchant Wholesalers"},{"NAICSCode":424480,"Description":"Fresh Fruit and Vegetable Merchant Wholesalers"},{"NAICSCode":424490,"Description":"Other Grocery and Related Products Merchant Wholesalers"},{"NAICSCode":424510,"Description":"Grain and Field Bean Merchant Wholesalers"},{"NAICSCode":424520,"Description":"Livestock Merchant Wholesalers"},{"NAICSCode":424590,"Description":"Other Farm Product Raw Material Merchant Wholesalers"},{"NAICSCode":424610,"Description":"Plastics Materials and Basic Forms and Shapes Merchant Wholesalers"},{"NAICSCode":424690,"Description":"Other Chemical and Allied Products Merchant Wholesalers"},{"NAICSCode":424710,"Description":"Petroleum Bulk Stations and Terminals"},{"NAICSCode":424720,"Description":"Petroleum and Petroleum Products Merchant Wholesalers (except Bulk Stations and Terminals)"},{"NAICSCode":424810,"Description":"Beer and Ale Merchant Wholesalers"},{"NAICSCode":424820,"Description":"Wine and Distilled Alcoholic Beverage Merchant Wholesalers"},{"NAICSCode":424910,"Description":"Farm Supplies Merchant Wholesalers"},{"NAICSCode":424920,"Description":"Book, Periodical, and Newspaper Merchant Wholesalers"},{"NAICSCode":424930,"Description":"Flower, Nursery Stock, and Florists' Supplies Merchant Wholesalers"},{"NAICSCode":424940,"Description":"Tobacco and Tobacco Product Merchant Wholesalers"},{"NAICSCode":424950,"Description":"Paint, Varnish, and Supplies Merchant Wholesalers"},{"NAICSCode":424990,"Description":"Other Miscellaneous Nondurable Goods Merchant Wholesalers"},{"NAICSCode":425110,"Description":"Business to Business Electronic Markets"},{"NAICSCode":425120,"Description":"Wholesale Trade Agents and Brokers"},{"NAICSCode":441110,"Description":"New Car Dealers"},{"NAICSCode":441120,"Description":"Used Car Dealers"},{"NAICSCode":441210,"Description":"Recreational Vehicle Dealers"},{"NAICSCode":441222,"Description":"Boat Dealers"},{"NAICSCode":441228,"Description":"Motorcycle, ATV, and All Other Motor Vehicle Dealers"},{"NAICSCode":441310,"Description":"Automotive Parts and Accessories Stores"},{"NAICSCode":441320,"Description":"Tire Dealers"},{"NAICSCode":442110,"Description":"Furniture Stores"},{"NAICSCode":442210,"Description":"Floor Covering Stores"},{"NAICSCode":442291,"Description":"Window Treatment Stores"},{"NAICSCode":442299,"Description":"All Other Home Furnishings Stores"},{"NAICSCode":443141,"Description":"Household Appliance Stores"},{"NAICSCode":443142,"Description":"Electronics Stores"},{"NAICSCode":444110,"Description":"Home Centers"},{"NAICSCode":444120,"Description":"Paint and Wallpaper Stores"},{"NAICSCode":444130,"Description":"Hardware Stores"},{"NAICSCode":444190,"Description":"Other Building Material Dealers"},{"NAICSCode":444210,"Description":"Outdoor Power Equipment Stores"},{"NAICSCode":444220,"Description":"Nursery, Garden Center, and Farm Supply Stores"},{"NAICSCode":445110,"Description":"Supermarkets and Other Grocery (except Convenience) Stores"},{"NAICSCode":445120,"Description":"Convenience Stores"},{"NAICSCode":445210,"Description":"Meat Markets"},{"NAICSCode":445220,"Description":"Fish and Seafood Markets"},{"NAICSCode":445230,"Description":"Fruit and Vegetable Markets"},{"NAICSCode":445291,"Description":"Baked Goods Stores"},{"NAICSCode":445292,"Description":"Confectionery and Nut Stores"},{"NAICSCode":445299,"Description":"All Other Specialty Food Stores"},{"NAICSCode":445310,"Description":"Beer, Wine, and Liquor Stores"},{"NAICSCode":446110,"Description":"Pharmacies and Drug Stores"},{"NAICSCode":446120,"Description":"Cosmetics, Beauty Supplies, and Perfume Stores"},{"NAICSCode":446130,"Description":"Optical Goods Stores"},{"NAICSCode":446191,"Description":"Food (Health) Supplement Stores"},{"NAICSCode":446199,"Description":"All Other Health and Personal Care Stores"},{"NAICSCode":447110,"Description":"Gasoline Stations with Convenience Stores"},{"NAICSCode":447190,"Description":"Other Gasoline Stations"},{"NAICSCode":448110,"Description":"Men's Clothing Stores"},{"NAICSCode":448120,"Description":"Women's Clothing Stores"},{"NAICSCode":448130,"Description":"Children's and Infants' Clothing Stores"},{"NAICSCode":448140,"Description":"Family Clothing Stores"},{"NAICSCode":448150,"Description":"Clothing Accessories Stores"},{"NAICSCode":448190,"Description":"Other Clothing Stores"},{"NAICSCode":448210,"Description":"Shoe Stores"},{"NAICSCode":448310,"Description":"Jewelry Stores"},{"NAICSCode":448320,"Description":"Luggage and Leather Goods Stores"},{"NAICSCode":451110,"Description":"Sporting Goods Stores"},{"NAICSCode":451120,"Description":"Hobby, Toy, and Game Stores"},{"NAICSCode":451130,"Description":"Sewing, Needlework, and Piece Goods Stores"},{"NAICSCode":451140,"Description":"Musical Instrument and Supplies Stores"},{"NAICSCode":451211,"Description":"Book Stores"},{"NAICSCode":451212,"Description":"News Dealers and Newsstands"},{"NAICSCode":452210,"Description":"Department Stores"},{"NAICSCode":452210,"Description":"Department Stores"},{"NAICSCode":452311,"Description":"Warehouse Clubs and Supercenters"},{"NAICSCode":452311,"Description":"Warehouse Clubs and Supercenters"},{"NAICSCode":452319,"Description":"All Other General Merchandise Stores"},{"NAICSCode":453110,"Description":"Florists"},{"NAICSCode":453210,"Description":"Office Supplies and Stationery Stores"},{"NAICSCode":453220,"Description":"Gift, Novelty, and Souvenir Stores"},{"NAICSCode":453310,"Description":"Used Merchandise Stores"},{"NAICSCode":453910,"Description":"Pet and Pet Supplies Stores"},{"NAICSCode":453920,"Description":"Art Dealers"},{"NAICSCode":453930,"Description":"Manufactured (Mobile) Home Dealers"},{"NAICSCode":453991,"Description":"Tobacco Stores"},{"NAICSCode":453998,"Description":"All Other Miscellaneous Store Retailers (except Tobacco Stores)"},{"NAICSCode":454110,"Description":"Electronic Shopping and Mail-Order Houses"},{"NAICSCode":454110,"Description":"Electronic Shopping and Mail-Order Houses"},{"NAICSCode":454110,"Description":"Electronic Shopping and Mail-Order Houses"},{"NAICSCode":454210,"Description":"Vending Machine Operators"},{"NAICSCode":454310,"Description":"Fuel Dealers"},{"NAICSCode":454390,"Description":"Other Direct Selling Establishments"},{"NAICSCode":481111,"Description":"Scheduled Passenger Air Transportation"},{"NAICSCode":481112,"Description":"Scheduled Freight Air Transportation"},{"NAICSCode":481211,"Description":"Nonscheduled Chartered Passenger Air Transportation"},{"NAICSCode":481212,"Description":"Nonscheduled Chartered Freight Air Transportation"},{"NAICSCode":481219,"Description":"Other Nonscheduled Air Transportation"},{"NAICSCode":482111,"Description":"Line-Haul Railroads"},{"NAICSCode":482112,"Description":"Short Line Railroads"},{"NAICSCode":483111,"Description":"Deep Sea Freight Transportation"},{"NAICSCode":483112,"Description":"Deep Sea Passenger Transportation"},{"NAICSCode":483113,"Description":"Coastal and Great Lakes Freight Transportation"},{"NAICSCode":483114,"Description":"Coastal and Great Lakes Passenger Transportation"},{"NAICSCode":483211,"Description":"Inland Water Freight Transportation"},{"NAICSCode":483212,"Description":"Inland Water Passenger Transportation"},{"NAICSCode":484110,"Description":"General Freight Trucking, Local"},{"NAICSCode":484121,"Description":"General Freight Trucking, Long-Distance, Truckload"},{"NAICSCode":484122,"Description":"General Freight Trucking, Long-Distance, Less Than Truckload"},{"NAICSCode":484210,"Description":"Used Household and Office Goods Moving"},{"NAICSCode":484220,"Description":"Specialized Freight (except Used Goods) Trucking, Local"},{"NAICSCode":484230,"Description":"Specialized Freight (except Used Goods) Trucking, Long-Distance"},{"NAICSCode":485111,"Description":"Mixed Mode Transit Systems"},{"NAICSCode":485112,"Description":"Commuter Rail Systems"},{"NAICSCode":485113,"Description":"Bus and Other Motor Vehicle Transit Systems"},{"NAICSCode":485119,"Description":"Other Urban Transit Systems"},{"NAICSCode":485210,"Description":"Interurban and Rural Bus Transportation"},{"NAICSCode":485310,"Description":"Taxi Service"},{"NAICSCode":485320,"Description":"Limousine Service"},{"NAICSCode":485410,"Description":"School and Employee Bus Transportation"},{"NAICSCode":485510,"Description":"Charter Bus Industry"},{"NAICSCode":485991,"Description":"Special Needs Transportation"},{"NAICSCode":485999,"Description":"All Other Transit and Ground Passenger Transportation"},{"NAICSCode":486110,"Description":"Pipeline Transportation of Crude Oil"},{"NAICSCode":486210,"Description":"Pipeline Transportation of Natural Gas"},{"NAICSCode":486910,"Description":"Pipeline Transportation of Refined Petroleum Products"},{"NAICSCode":486990,"Description":"All Other Pipeline Transportation"},{"NAICSCode":487110,"Description":"Scenic and Sightseeing Transportation, Land"},{"NAICSCode":487210,"Description":"Scenic and Sightseeing Transportation, Water"},{"NAICSCode":487990,"Description":"Scenic and Sightseeing Transportation, Other"},{"NAICSCode":488111,"Description":"Air Traffic Control"},{"NAICSCode":488119,"Description":"Other Airport Operations"},{"NAICSCode":488190,"Description":"Other Support Activities for Air Transportation"},{"NAICSCode":488210,"Description":"Support Activities for Rail Transportation"},{"NAICSCode":488310,"Description":"Port and Harbor Operations"},{"NAICSCode":488320,"Description":"Marine Cargo Handling"},{"NAICSCode":488330,"Description":"Navigational Services to Shipping"},{"NAICSCode":488390,"Description":"Other Support Activities for Water Transportation"},{"NAICSCode":488410,"Description":"Motor Vehicle Towing"},{"NAICSCode":488490,"Description":"Other Support Activities for Road Transportation"},{"NAICSCode":488510,"Description":"Freight Transportation Arrangement"},{"NAICSCode":488991,"Description":"Packing and Crating"},{"NAICSCode":488999,"Description":"All Other Support Activities for Transportation"},{"NAICSCode":491110,"Description":"Postal Service"},{"NAICSCode":492110,"Description":"Couriers and Express Delivery Services"},{"NAICSCode":492210,"Description":"Local Messengers and Local Delivery"},{"NAICSCode":493110,"Description":"General Warehousing and Storage"},{"NAICSCode":493120,"Description":"Refrigerated Warehousing and Storage"},{"NAICSCode":493130,"Description":"Farm Product Warehousing and Storage"},{"NAICSCode":493190,"Description":"Other Warehousing and Storage"},{"NAICSCode":511110,"Description":"Newspaper Publishers"},{"NAICSCode":511120,"Description":"Periodical Publishers"},{"NAICSCode":511130,"Description":"Book Publishers"},{"NAICSCode":511140,"Description":"Directory and Mailing List Publishers"},{"NAICSCode":511191,"Description":"Greeting Card Publishers"},{"NAICSCode":511199,"Description":"All Other Publishers"},{"NAICSCode":511210,"Description":"Software Publishers"},{"NAICSCode":512110,"Description":"Motion Picture and Video Production"},{"NAICSCode":512120,"Description":"Motion Picture and Video Distribution"},{"NAICSCode":512131,"Description":"Motion Picture Theaters (except Drive-Ins)"},{"NAICSCode":512132,"Description":"Drive-In Motion Picture Theaters"},{"NAICSCode":512191,"Description":"Teleproduction and Other Postproduction Services"},{"NAICSCode":512199,"Description":"Other Motion Picture and Video Industries"},{"NAICSCode":512230,"Description":"Music Publishers"},{"NAICSCode":512240,"Description":"Sound Recording Studios"},{"NAICSCode":512250,"Description":"Record Production and Distribution"},{"NAICSCode":512250,"Description":"Record Production and Distribution"},{"NAICSCode":512290,"Description":"Other Sound Recording Industries"},{"NAICSCode":515111,"Description":"Radio Networks"},{"NAICSCode":515112,"Description":"Radio Stations"},{"NAICSCode":515120,"Description":"Television Broadcasting"},{"NAICSCode":515210,"Description":"Cable and Other Subscription Programming"},{"NAICSCode":517311,"Description":"Wired Telecommunications Carriers"},{"NAICSCode":517312,"Description":"Wireless Telecommunications Carriers (except Satellite)"},{"NAICSCode":517410,"Description":"Satellite Telecommunications"},{"NAICSCode":517911,"Description":"Telecommunications Resellers"},{"NAICSCode":517919,"Description":"All Other Telecommunications"},{"NAICSCode":518210,"Description":"Data Processing, Hosting, and Related Services"},{"NAICSCode":519110,"Description":"News Syndicates"},{"NAICSCode":519120,"Description":"Libraries and Archives"},{"NAICSCode":519130,"Description":"Internet Publishing and Broadcasting and Web Search Portals"},{"NAICSCode":519190,"Description":"All Other Information Services"},{"NAICSCode":521110,"Description":"Monetary Authorities-Central Bank"},{"NAICSCode":522110,"Description":"Commercial Banking"},{"NAICSCode":522120,"Description":"Savings Institutions"},{"NAICSCode":522130,"Description":"Credit Unions"},{"NAICSCode":522190,"Description":"Other Depository Credit Intermediation"},{"NAICSCode":522210,"Description":"Credit Card Issuing"},{"NAICSCode":522220,"Description":"Sales Financing"},{"NAICSCode":522291,"Description":"Consumer Lending"},{"NAICSCode":522292,"Description":"Real Estate Credit"},{"NAICSCode":522293,"Description":"International Trade Financing"},{"NAICSCode":522294,"Description":"Secondary Market Financing"},{"NAICSCode":522298,"Description":"All Other Nondepository Credit Intermediation"},{"NAICSCode":522310,"Description":"Mortgage and Nonmortgage Loan Brokers"},{"NAICSCode":522320,"Description":"Financial Transactions Processing, Reserve, and Clearinghouse Activities"},{"NAICSCode":522390,"Description":"Other Activities Related to Credit Intermediation"},{"NAICSCode":523110,"Description":"Investment Banking and Securities Dealing"},{"NAICSCode":523120,"Description":"Securities Brokerage"},{"NAICSCode":523130,"Description":"Commodity Contracts Dealing"},{"NAICSCode":523140,"Description":"Commodity Contracts Brokerage"},{"NAICSCode":523210,"Description":"Securities and Commodity Exchanges"},{"NAICSCode":523910,"Description":"Miscellaneous Intermediation"},{"NAICSCode":523920,"Description":"Portfolio Management"},{"NAICSCode":523930,"Description":"Investment Advice"},{"NAICSCode":523991,"Description":"Trust, Fiduciary, and Custody Activities"},{"NAICSCode":523999,"Description":"Miscellaneous Financial Investment Activities"},{"NAICSCode":524113,"Description":"Direct Life Insurance Carriers"},{"NAICSCode":524114,"Description":"Direct Health and Medical Insurance Carriers"},{"NAICSCode":524126,"Description":"Direct Property and Casualty Insurance Carriers"},{"NAICSCode":524127,"Description":"Direct Title Insurance Carriers"},{"NAICSCode":524128,"Description":"Other Direct Insurance (except Life, Health, and Medical) Carriers"},{"NAICSCode":524130,"Description":"Reinsurance Carriers"},{"NAICSCode":524210,"Description":"Insurance Agencies and Brokerages"},{"NAICSCode":524291,"Description":"Claims Adjusting"},{"NAICSCode":524292,"Description":"Third Party Administration of Insurance and Pension Funds"},{"NAICSCode":524298,"Description":"All Other Insurance Related Activities"},{"NAICSCode":525110,"Description":"Pension Funds"},{"NAICSCode":525120,"Description":"Health and Welfare Funds"},{"NAICSCode":525190,"Description":"Other Insurance Funds"},{"NAICSCode":525910,"Description":"Open-End Investment Funds"},{"NAICSCode":525920,"Description":"Trusts, Estates, and Agency Accounts"},{"NAICSCode":525990,"Description":"Other Financial Vehicles"},{"NAICSCode":531110,"Description":"Lessors of Residential Buildings and Dwellings"},{"NAICSCode":531120,"Description":"Lessors of Nonresidential Buildings (except Miniwarehouses)"},{"NAICSCode":531130,"Description":"Lessors of Miniwarehouses and Self-Storage Units"},{"NAICSCode":531190,"Description":"Lessors of Other Real Estate Property"},{"NAICSCode":531210,"Description":"Offices of Real Estate Agents and Brokers"},{"NAICSCode":531311,"Description":"Residential Property Managers"},{"NAICSCode":531312,"Description":"Nonresidential Property Managers"},{"NAICSCode":531320,"Description":"Offices of Real Estate Appraisers"},{"NAICSCode":531390,"Description":"Other Activities Related to Real Estate"},{"NAICSCode":532111,"Description":"Passenger Car Rental"},{"NAICSCode":532112,"Description":"Passenger Car Leasing"},{"NAICSCode":532120,"Description":"Truck, Utility Trailer, and RV (Recreational Vehicle) Rental and Leasing"},{"NAICSCode":532210,"Description":"Consumer Electronics and Appliances Rental"},{"NAICSCode":532281,"Description":"Formal Wear and Costume Rental"},{"NAICSCode":532282,"Description":"Video Tape and Disc Rental"},{"NAICSCode":532283,"Description":"Home Health Equipment Rental"},{"NAICSCode":532284,"Description":"Recreational Goods Rental"},{"NAICSCode":532289,"Description":"All Other Consumer Goods Rental"},{"NAICSCode":532310,"Description":"General Rental Centers"},{"NAICSCode":532411,"Description":"Commercial Air, Rail, and Water Transportation Equipment Rental and Leasing"},{"NAICSCode":532412,"Description":"Construction, Mining, and Forestry Machinery and Equipment Rental and Leasing"},{"NAICSCode":532420,"Description":"Office Machinery and Equipment Rental and Leasing"},{"NAICSCode":532490,"Description":"Other Commercial and Industrial Machinery and Equipment Rental and Leasing"},{"NAICSCode":533110,"Description":"Lessors of Nonfinancial Intangible Assets (except Copyrighted Works)"},{"NAICSCode":541110,"Description":"Offices of Lawyers"},{"NAICSCode":541120,"Description":"Offices of Notaries"},{"NAICSCode":541191,"Description":"Title Abstract and Settlement Offices"},{"NAICSCode":541199,"Description":"All Other Legal Services"},{"NAICSCode":541211,"Description":"Offices of Certified Public Accountants"},{"NAICSCode":541213,"Description":"Tax Preparation Services"},{"NAICSCode":541214,"Description":"Payroll Services"},{"NAICSCode":541219,"Description":"Other Accounting Services"},{"NAICSCode":541310,"Description":"Architectural Services"},{"NAICSCode":541320,"Description":"Landscape Architectural Services"},{"NAICSCode":541330,"Description":"Engineering Services"},{"NAICSCode":541340,"Description":"Drafting Services"},{"NAICSCode":541350,"Description":"Building Inspection Services"},{"NAICSCode":541360,"Description":"Geophysical Surveying and Mapping Services"},{"NAICSCode":541370,"Description":"Surveying and Mapping (except Geophysical) Services"},{"NAICSCode":541380,"Description":"Testing Laboratories"},{"NAICSCode":541410,"Description":"Interior Design Services"},{"NAICSCode":541420,"Description":"Industrial Design Services"},{"NAICSCode":541430,"Description":"Graphic Design Services"},{"NAICSCode":541490,"Description":"Other Specialized Design Services"},{"NAICSCode":541511,"Description":"Custom Computer Programming Services"},{"NAICSCode":541512,"Description":"Computer Systems Design Services"},{"NAICSCode":541513,"Description":"Computer Facilities Management Services"},{"NAICSCode":541519,"Description":"Other Computer Related Services"},{"NAICSCode":541611,"Description":"Administrative Management and General Management Consulting Services"},{"NAICSCode":541612,"Description":"Human Resources Consulting Services"},{"NAICSCode":541613,"Description":"Marketing Consulting Services"},{"NAICSCode":541614,"Description":"Process, Physical Distribution, and Logistics Consulting Services"},{"NAICSCode":541618,"Description":"Other Management Consulting Services"},{"NAICSCode":541620,"Description":"Environmental Consulting Services"},{"NAICSCode":541690,"Description":"Other Scientific and Technical Consulting Services"},{"NAICSCode":541713,"Description":"Research and Development in Nanotechnology"},{"NAICSCode":541713,"Description":"Research and Development in Nanotechnology"},{"NAICSCode":541714,"Description":"Research and Development in Biotechnology (except Nanobiotechnology)"},{"NAICSCode":541715,"Description":"Research and Development in the Physical, Engineering, and Life Sciences (except Nanotechnology and Biotechnology)"},{"NAICSCode":541720,"Description":"Research and Development in the Social Sciences and Humanities"},{"NAICSCode":541810,"Description":"Advertising Agencies"},{"NAICSCode":541820,"Description":"Public Relations Agencies"},{"NAICSCode":541830,"Description":"Media Buying Agencies"},{"NAICSCode":541840,"Description":"Media Representatives"},{"NAICSCode":541850,"Description":"Outdoor Advertising"},{"NAICSCode":541860,"Description":"Direct Mail Advertising"},{"NAICSCode":541870,"Description":"Advertising Material Distribution Services"},{"NAICSCode":541890,"Description":"Other Services Related to Advertising"},{"NAICSCode":541910,"Description":"Marketing Research and Public Opinion Polling"},{"NAICSCode":541921,"Description":"Photography Studios, Portrait"},{"NAICSCode":541922,"Description":"Commercial Photography"},{"NAICSCode":541930,"Description":"Translation and Interpretation Services"},{"NAICSCode":541940,"Description":"Veterinary Services"},{"NAICSCode":541990,"Description":"All Other Professional, Scientific, and Technical Services"},{"NAICSCode":551111,"Description":"Offices of Bank Holding Companies"},{"NAICSCode":551112,"Description":"Offices of Other Holding Companies"},{"NAICSCode":551114,"Description":"Corporate, Subsidiary, and Regional Managing Offices"},{"NAICSCode":561110,"Description":"Office Administrative Services"},{"NAICSCode":561210,"Description":"Facilities Support Services"},{"NAICSCode":561311,"Description":"Employment Placement Agencies"},{"NAICSCode":561312,"Description":"Executive Search Services"},{"NAICSCode":561320,"Description":"Temporary Help Services"},{"NAICSCode":561330,"Description":"Professional Employer Organizations"},{"NAICSCode":561410,"Description":"Document Preparation Services"},{"NAICSCode":561421,"Description":"Telephone Answering Services"},{"NAICSCode":561422,"Description":"Telemarketing Bureaus and Other Contact Centers"},{"NAICSCode":561431,"Description":"Private Mail Centers"},{"NAICSCode":561439,"Description":"Other Business Service Centers (including Copy Shops)"},{"NAICSCode":561440,"Description":"Collection Agencies"},{"NAICSCode":561450,"Description":"Credit Bureaus"},{"NAICSCode":561491,"Description":"Repossession Services"},{"NAICSCode":561492,"Description":"Court Reporting and Stenotype Services"},{"NAICSCode":561499,"Description":"All Other Business Support Services"},{"NAICSCode":561510,"Description":"Travel Agencies"},{"NAICSCode":561520,"Description":"Tour Operators"},{"NAICSCode":561591,"Description":"Convention and Visitors Bureaus"},{"NAICSCode":561599,"Description":"All Other Travel Arrangement and Reservation Services"},{"NAICSCode":561611,"Description":"Investigation Services"},{"NAICSCode":561612,"Description":"Security Guards and Patrol Services"},{"NAICSCode":561613,"Description":"Armored Car Services"},{"NAICSCode":561621,"Description":"Security Systems Services (except Locksmiths)"},{"NAICSCode":561622,"Description":"Locksmiths"},{"NAICSCode":561710,"Description":"Exterminating and Pest Control Services"},{"NAICSCode":561720,"Description":"Janitorial Services"},{"NAICSCode":561730,"Description":"Landscaping Services"},{"NAICSCode":561740,"Description":"Carpet and Upholstery Cleaning Services"},{"NAICSCode":561790,"Description":"Other Services to Buildings and Dwellings"},{"NAICSCode":561910,"Description":"Packaging and Labeling Services"},{"NAICSCode":561920,"Description":"Convention and Trade Show Organizers"},{"NAICSCode":561990,"Description":"All Other Support Services"},{"NAICSCode":562111,"Description":"Solid Waste Collection"},{"NAICSCode":562112,"Description":"Hazardous Waste Collection"},{"NAICSCode":562119,"Description":"Other Waste Collection"},{"NAICSCode":562211,"Description":"Hazardous Waste Treatment and Disposal"},{"NAICSCode":562212,"Description":"Solid Waste Landfill"},{"NAICSCode":562213,"Description":"Solid Waste Combustors and Incinerators"},{"NAICSCode":562219,"Description":"Other Nonhazardous Waste Treatment and Disposal"},{"NAICSCode":562910,"Description":"Remediation Services"},{"NAICSCode":562920,"Description":"Materials Recovery Facilities"},{"NAICSCode":562991,"Description":"Septic Tank and Related Services"},{"NAICSCode":562998,"Description":"All Other Miscellaneous Waste Management Services"},{"NAICSCode":611110,"Description":"Elementary and Secondary Schools"},{"NAICSCode":611210,"Description":"Junior Colleges"},{"NAICSCode":611310,"Description":"Colleges, Universities, and Professional Schools"},{"NAICSCode":611410,"Description":"Business and Secretarial Schools"},{"NAICSCode":611420,"Description":"Computer Training"},{"NAICSCode":611430,"Description":"Professional and Management Development Training"},{"NAICSCode":611511,"Description":"Cosmetology and Barber Schools"},{"NAICSCode":611512,"Description":"Flight Training"},{"NAICSCode":611513,"Description":"Apprenticeship Training"},{"NAICSCode":611519,"Description":"Other Technical and Trade Schools"},{"NAICSCode":611610,"Description":"Fine Arts Schools"},{"NAICSCode":611620,"Description":"Sports and Recreation Instruction"},{"NAICSCode":611630,"Description":"Language Schools"},{"NAICSCode":611691,"Description":"Exam Preparation and Tutoring"},{"NAICSCode":611692,"Description":"Automobile Driving Schools"},{"NAICSCode":611699,"Description":"All Other Miscellaneous Schools and Instruction"},{"NAICSCode":611710,"Description":"Educational Support Services"},{"NAICSCode":621111,"Description":"Offices of Physicians (except Mental Health Specialists)"},{"NAICSCode":621112,"Description":"Offices of Physicians, Mental Health Specialists"},{"NAICSCode":621210,"Description":"Offices of Dentists"},{"NAICSCode":621310,"Description":"Offices of Chiropractors"},{"NAICSCode":621320,"Description":"Offices of Optometrists"},{"NAICSCode":621330,"Description":"Offices of Mental Health Practitioners (except Physicians)"},{"NAICSCode":621340,"Description":"Offices of Physical, Occupational and Speech Therapists, and Audiologists"},{"NAICSCode":621391,"Description":"Offices of Podiatrists"},{"NAICSCode":621399,"Description":"Offices of All Other Miscellaneous Health Practitioners"},{"NAICSCode":621410,"Description":"Family Planning Centers"},{"NAICSCode":621420,"Description":"Outpatient Mental Health and Substance Abuse Centers"},{"NAICSCode":621491,"Description":"HMO Medical Centers"},{"NAICSCode":621492,"Description":"Kidney Dialysis Centers"},{"NAICSCode":621493,"Description":"Freestanding Ambulatory Surgical and Emergency Centers"},{"NAICSCode":621498,"Description":"All Other Outpatient Care Centers"},{"NAICSCode":621511,"Description":"Medical Laboratories"},{"NAICSCode":621512,"Description":"Diagnostic Imaging Centers"},{"NAICSCode":621610,"Description":"Home Health Care Services"},{"NAICSCode":621910,"Description":"Ambulance Services"},{"NAICSCode":621991,"Description":"Blood and Organ Banks"},{"NAICSCode":621999,"Description":"All Other Miscellaneous Ambulatory Health Care Services"},{"NAICSCode":622110,"Description":"General Medical and Surgical Hospitals"},{"NAICSCode":622210,"Description":"Psychiatric and Substance Abuse Hospitals"},{"NAICSCode":622310,"Description":"Specialty (except Psychiatric and Substance Abuse) Hospitals"},{"NAICSCode":623110,"Description":"Nursing Care Facilities (Skilled Nursing Facilities)"},{"NAICSCode":623210,"Description":"Residential Intellectual and Developmental Disability Facilities"},{"NAICSCode":623220,"Description":"Residential Mental Health and Substance Abuse Facilities"},{"NAICSCode":623311,"Description":"Continuing Care Retirement Communities"},{"NAICSCode":623312,"Description":"Assisted Living Facilities for the Elderly"},{"NAICSCode":623990,"Description":"Other Residential Care Facilities"},{"NAICSCode":624110,"Description":"Child and Youth Services"},{"NAICSCode":624120,"Description":"Services for the Elderly and Persons with Disabilities"},{"NAICSCode":624190,"Description":"Other Individual and Family Services"},{"NAICSCode":624210,"Description":"Community Food Services"},{"NAICSCode":624221,"Description":"Temporary Shelters"},{"NAICSCode":624229,"Description":"Other Community Housing Services"},{"NAICSCode":624230,"Description":"Emergency and Other Relief Services"},{"NAICSCode":624310,"Description":"Vocational Rehabilitation Services"},{"NAICSCode":624410,"Description":"Child Day Care Services"},{"NAICSCode":711110,"Description":"Theater Companies and Dinner Theaters"},{"NAICSCode":711120,"Description":"Dance Companies"},{"NAICSCode":711130,"Description":"Musical Groups and Artists"},{"NAICSCode":711190,"Description":"Other Performing Arts Companies"},{"NAICSCode":711211,"Description":"Sports Teams and Clubs"},{"NAICSCode":711212,"Description":"Racetracks"},{"NAICSCode":711219,"Description":"Other Spectator Sports"},{"NAICSCode":711310,"Description":"Promoters of Performing Arts, Sports, and Similar Events with Facilities"},{"NAICSCode":711320,"Description":"Promoters of Performing Arts, Sports, and Similar Events without Facilities"},{"NAICSCode":711410,"Description":"Agents and Managers for Artists, Athletes, Entertainers, and Other Public Figures"},{"NAICSCode":711510,"Description":"Independent Artists, Writers, and Performers"},{"NAICSCode":712110,"Description":"Museums"},{"NAICSCode":712120,"Description":"Historical Sites"},{"NAICSCode":712130,"Description":"Zoos and Botanical Gardens"},{"NAICSCode":712190,"Description":"Nature Parks and Other Similar Institutions"},{"NAICSCode":713110,"Description":"Amusement and Theme Parks"},{"NAICSCode":713120,"Description":"Amusement Arcades"},{"NAICSCode":713210,"Description":"Casinos (except Casino Hotels)"},{"NAICSCode":713290,"Description":"Other Gambling Industries"},{"NAICSCode":713910,"Description":"Golf Courses and Country Clubs"},{"NAICSCode":713920,"Description":"Skiing Facilities"},{"NAICSCode":713930,"Description":"Marinas"},{"NAICSCode":713940,"Description":"Fitness and Recreational Sports Centers"},{"NAICSCode":713950,"Description":"Bowling Centers"},{"NAICSCode":713990,"Description":"All Other Amusement and Recreation Industries"},{"NAICSCode":721110,"Description":"Hotels (except Casino Hotels) and Motels"},{"NAICSCode":721120,"Description":"Casino Hotels"},{"NAICSCode":721191,"Description":"Bed-and-Breakfast Inns"},{"NAICSCode":721199,"Description":"All Other Traveler Accommodation"},{"NAICSCode":721211,"Description":"RV (Recreational Vehicle) Parks and Campgrounds"},{"NAICSCode":721214,"Description":"Recreational and Vacation Camps (except Campgrounds)"},{"NAICSCode":721310,"Description":"Rooming and Boarding Houses, Dormitories, and Workers' Camps"},{"NAICSCode":722310,"Description":"Food Service Contractors"},{"NAICSCode":722320,"Description":"Caterers"},{"NAICSCode":722330,"Description":"Mobile Food Services"},{"NAICSCode":722410,"Description":"Drinking Places (Alcoholic Beverages)"},{"NAICSCode":722511,"Description":"Full-Service Restaurants"},{"NAICSCode":722513,"Description":"Limited-Service Restaurants"},{"NAICSCode":722514,"Description":"Cafeterias, Grill Buffets, and Buffets"},{"NAICSCode":722515,"Description":"Snack and Nonalcoholic Beverage Bars"},{"NAICSCode":811111,"Description":"General Automotive Repair"},{"NAICSCode":811112,"Description":"Automotive Exhaust System Repair"},{"NAICSCode":811113,"Description":"Automotive Transmission Repair"},{"NAICSCode":811118,"Description":"Other Automotive Mechanical and Electrical Repair and Maintenance"},{"NAICSCode":811121,"Description":"Automotive Body, Paint, and Interior Repair and Maintenance"},{"NAICSCode":811122,"Description":"Automotive Glass Replacement Shops"},{"NAICSCode":811191,"Description":"Automotive Oil Change and Lubrication Shops"},{"NAICSCode":811192,"Description":"Car Washes"},{"NAICSCode":811198,"Description":"All Other Automotive Repair and Maintenance"},{"NAICSCode":811211,"Description":"Consumer Electronics Repair and Maintenance"},{"NAICSCode":811212,"Description":"Computer and Office Machine Repair and Maintenance"},{"NAICSCode":811213,"Description":"Communication Equipment Repair and Maintenance"},{"NAICSCode":811219,"Description":"Other Electronic and Precision Equipment Repair and Maintenance"},{"NAICSCode":811310,"Description":"Commercial and Industrial Machinery and Equipment (except Automotive and Electronic) Repair and Maintenance"},{"NAICSCode":811411,"Description":"Home and Garden Equipment Repair and Maintenance"},{"NAICSCode":811412,"Description":"Appliance Repair and Maintenance"},{"NAICSCode":811420,"Description":"Reupholstery and Furniture Repair"},{"NAICSCode":811430,"Description":"Footwear and Leather Goods Repair"},{"NAICSCode":811490,"Description":"Other Personal and Household Goods Repair and Maintenance"},{"NAICSCode":812111,"Description":"Barber Shops"},{"NAICSCode":812112,"Description":"Beauty Salons"},{"NAICSCode":812113,"Description":"Nail Salons"},{"NAICSCode":812191,"Description":"Diet and Weight Reducing Centers"},{"NAICSCode":812199,"Description":"Other Personal Care Services"},{"NAICSCode":812210,"Description":"Funeral Homes and Funeral Services"},{"NAICSCode":812220,"Description":"Cemeteries and Crematories"},{"NAICSCode":812310,"Description":"Coin-Operated Laundries and Drycleaners"},{"NAICSCode":812320,"Description":"Drycleaning and Laundry Services (except Coin-Operated)"},{"NAICSCode":812331,"Description":"Linen Supply"},{"NAICSCode":812332,"Description":"Industrial Launderers"},{"NAICSCode":812910,"Description":"Pet Care (except Veterinary) Services"},{"NAICSCode":812921,"Description":"Photofinishing Laboratories (except One-Hour)"},{"NAICSCode":812922,"Description":"One-Hour Photofinishing"},{"NAICSCode":812930,"Description":"Parking Lots and Garages"},{"NAICSCode":812990,"Description":"All Other Personal Services"},{"NAICSCode":813110,"Description":"Religious Organizations"},{"NAICSCode":813211,"Description":"Grantmaking Foundations"},{"NAICSCode":813212,"Description":"Voluntary Health Organizations"},{"NAICSCode":813219,"Description":"Other Grantmaking and Giving Services"},{"NAICSCode":813311,"Description":"Human Rights Organizations"},{"NAICSCode":813312,"Description":"Environment, Conservation and Wildlife Organizations"},{"NAICSCode":813319,"Description":"Other Social Advocacy Organizations"},{"NAICSCode":813410,"Description":"Civic and Social Organizations"},{"NAICSCode":813910,"Description":"Business Associations"},{"NAICSCode":813920,"Description":"Professional Organizations"},{"NAICSCode":813930,"Description":"Labor Unions and Similar Labor Organizations"},{"NAICSCode":813940,"Description":"Political Organizations"},{"NAICSCode":813990,"Description":"Other Similar Organizations (except Business, Professional, Labor, and Political Organizations)"},{"NAICSCode":814110,"Description":"Private Households"},{"NAICSCode":921110,"Description":"Executive Offices"},{"NAICSCode":921120,"Description":"Legislative Bodies"},{"NAICSCode":921130,"Description":"Public Finance Activities"},{"NAICSCode":921140,"Description":"Executive and Legislative Offices, Combined"},{"NAICSCode":921150,"Description":"American Indian and Alaska Native Tribal Governments"},{"NAICSCode":921190,"Description":"Other General Government Support"},{"NAICSCode":922110,"Description":"Courts"},{"NAICSCode":922120,"Description":"Police Protection"},{"NAICSCode":922130,"Description":"Legal Counsel and Prosecution"},{"NAICSCode":922140,"Description":"Correctional Institutions"},{"NAICSCode":922150,"Description":"Parole Offices and Probation Offices"},{"NAICSCode":922160,"Description":"Fire Protection"},{"NAICSCode":922190,"Description":"Other Justice, Public Order, and Safety Activities"},{"NAICSCode":923110,"Description":"Administration of Education Programs"},{"NAICSCode":923120,"Description":"Administration of Public Health Programs"},{"NAICSCode":923130,"Description":"Administration of Human Resource Programs (except Education, Public Health, and Veterans' Affairs Programs)"},{"NAICSCode":923140,"Description":"Administration of Veterans' Affairs"},{"NAICSCode":924110,"Description":"Administration of Air and Water Resource and Solid Waste Management Programs"},{"NAICSCode":924120,"Description":"Administration of Conservation Programs"},{"NAICSCode":925110,"Description":"Administration of Housing Programs"},{"NAICSCode":925120,"Description":"Administration of Urban Planning and Community and Rural Development"},{"NAICSCode":926110,"Description":"Administration of General Economic Programs"},{"NAICSCode":926120,"Description":"Regulation and Administration of Transportation Programs"},{"NAICSCode":926130,"Description":"Regulation and Administration of Communications, Electric, Gas, and Other Utilities"},{"NAICSCode":926140,"Description":"Regulation of Agricultural Marketing and Commodities"},{"NAICSCode":926150,"Description":"Regulation, Licensing, and Inspection of Miscellaneous Commercial Sectors"},{"NAICSCode":927110,"Description":"Space Research and Technology"},{"NAICSCode":928110,"Description":"National Security"},{"NAICSCode":928120,"Description":"International Affairs"}]




In [10]:
#@title Download PPP data
#list of data URLs
PPPData = [
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/501af711-1c91-477a-80ce-bf6428eb9253/download/public_150k_plus_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/85f66605-568a-4400-aa5c-3803873153dc/download/public_up_to_150k_1_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/783414f9-888b-47fb-b5dc-b1c0b102b6e5/download/public_up_to_150k_2_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/148d5da2-c802-4c78-836e-6ee1d6c2bdf9/download/public_up_to_150k_3_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/0fac3770-49da-4160-ae9f-53fc1b791c6e/download/public_up_to_150k_4_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/b4ceb5fa-7693-43dc-af70-837cd88d84e9/download/public_up_to_150k_5_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/980ec35b-d98c-452f-b4fc-2677dc923b90/download/public_up_to_150k_6_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/61ef84ba-4e7c-4d19-96c9-761daed984d0/download/public_up_to_150k_7_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/ba418afb-582d-444e-bf16-f73a9c0fea75/download/public_up_to_150k_8_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/08de1369-982f-48a1-a828-256cb9ef488c/download/public_up_to_150k_9_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/482b116e-35a6-4cd7-9171-e9dbe8b906eb/download/public_up_to_150k_10_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/2c2adf29-a57c-4696-bfbf-452acd021eca/download/public_up_to_150k_11_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/0950ec12-6edc-4df3-acfc-b53a892e220d/download/public_up_to_150k_12_220403.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/aab8e9f9-36d1-42e1-b3ba-e59c79f1d7f0/download/ppp-data-dictionary.xlsx"
]

#download all csvs
for file in PPPData:
    if (os.path.exists(f'{WORKING_PATH}/{file.split("/")[-1]}.csv')==False):
        with urllib.request.urlopen(file ) as dataFile, open (f'{WORKING_PATH}/{file.split("/")[-1]}.csv', "w") as f:
            f.write(dataFile.read().decode())


In [22]:
#@title FILTER
#@markdown #filter selection

dataFrameName = 'KYSystemDesignServices' #@param 
selected_attribute = 'CD' #@param  ["None","LoanNumber", "DateApproved", "SBAOfficeCode", "ProcessingMethod", "BorrowerName", "BorrowerAddress", "BorrowerCity", "BorrowerState", "BorrowerZip", "LoanStatusDate", "LoanStatus", "Term", "SBAGuarantyPercentage", "InitialApprovalAmount", "CurrentApprovalAmount", "UndisbursedAmount", "FranchiseName", "ServicingLenderLocationID", "ServicingLenderName", "ServicingLenderAddress", "ServicingLenderCity", "ServicingLenderState", "ServicingLenderZip", "RuralUrbanIndicator", "HubzoneIndicator", "LMIIndicator", "BusinessAgeDescription", "ProjectCity", "ProjectCountyName", "ProjectState", "ProjectZip", "CD", "JobsReported", "NAICSCode", "Race", "Ethnicity", "UTILITIES_PROCEED", "PAYROLL_PROCEED", "MORTGAGE_INTEREST_PROCEED", "RENT_PROCEED", "REFINANCE_EIDL_PROCEED", "HEALTH_CARE_PROCEED", "DEBT_INTEREST_PROCEED", "BusinessType", "OriginatingLenderLocationID", "OriginatingLender", "OriginatingLenderCity", "OriginatingLenderState", "Gender", "Veteran", "NonProfit", "ForgivenessAmount", "ForgivenessDate"]
acceptable_values = ['KY-06', 'KY-01', 'KY-02', 'KY-03', 'KY-04', 'KY-05'] #@param 
naics_code_descriptions = 'Computer Systems Design Services' #@param  ["None", "Soybean Farming", "Oilseed (except Soybean) Farming", "Dry Pea and Bean Farming", "Wheat Farming", "Corn Farming", "Rice Farming", "Oilseed and Grain Combination Farming", "All Other Grain Farming", "Potato Farming", "Other Vegetable (except Potato) and Melon Farming", "Orange Groves", "Citrus (except Orange) Groves", "Apple Orchards", "Grape Vineyards", "Strawberry Farming", "Berry (except Strawberry) Farming", "Tree Nut Farming", "Fruit and Tree Nut Combination Farming", "Other Noncitrus Fruit Farming", "Mushroom Production", "Other Food Crops Grown Under Cover", "Nursery and Tree Production", "Floriculture Production", "Tobacco Farming", "Cotton Farming", "Sugarcane Farming", "Hay Farming", "Sugar Beet Farming", "Peanut Farming", "All Other Miscellaneous Crop Farming", "Beef Cattle Ranching and Farming", "Cattle Feedlots", "Dairy Cattle and Milk Production", "Dual-Purpose Cattle Ranching and Farming", "Hog and Pig Farming", "Chicken Egg Production", "Broilers and Other Meat Type Chicken Production", "Turkey Production", "Poultry Hatcheries", "Other Poultry Production", "Sheep Farming", "Goat Farming", "Finfish Farming and Fish Hatcheries", "Shellfish Farming", "Other Aquaculture", "Apiculture", "Horses and Other Equine Production", "Fur-Bearing Animal and Rabbit Production", "All Other Animal Production", "Timber Tract Operations", "Forest Nurseries and Gathering of Forest Products", "Logging", "Finfish Fishing", "Shellfish Fishing", "Other Marine Fishing", "Hunting and Trapping", "Cotton Ginning", "Soil Preparation, Planting, and Cultivating", "Crop Harvesting, Primarily by Machine", "Postharvest Crop Activities (except Cotton Ginning)", "Farm Labor Contractors and Crew Leaders", "Farm Management Services", "Support Activities for Animal Production", "Support Activities for Forestry", "Crude Petroleum Extraction", "Natural Gas Extraction", "Natural Gas Extraction", "Bituminous Coal and Lignite Surface Mining", "Bituminous Coal Underground Mining", "Anthracite Mining", "Iron Ore Mining", "Gold Ore Mining", "Silver Ore Mining", "Copper, Nickel, Lead, and Zinc Mining", "Copper, Nickel, Lead, and Zinc Mining", "Uranium-Radium-Vanadium Ore Mining", "All Other Metal Ore Mining", "Dimension Stone Mining and Quarrying", "Crushed and Broken Limestone Mining and Quarrying", "Crushed and Broken Granite Mining and Quarrying", "Other Crushed and Broken Stone Mining and Quarrying", "Construction Sand and Gravel Mining", "Industrial Sand Mining", "Kaolin and Ball Clay Mining", "Clay and Ceramic and Refractory Minerals Mining", "Potash, Soda, and Borate Mineral Mining", "Phosphate Rock Mining", "Other Chemical and Fertilizer Mineral Mining", "All Other Nonmetallic Mineral Mining", "Drilling Oil and Gas Wells", "Support Activities for Oil and Gas Operations", "Support Activities for Coal Mining", "Support Activities for Metal Mining", "Support Activities for Nonmetallic Minerals (except Fuels) Mining", "Hydroelectric Power Generation", "Fossil Fuel Electric Power Generation", "Nuclear Electric Power Generation", "Solar Electric Power Generation", "Wind Electric Power Generation", "Geothermal Electric Power Generation", "Biomass Electric Power Generation", "Other Electric Power Generation", "Electric Bulk Power Transmission and Control", "Electric Power Distribution", "Natural Gas Distribution", "Water Supply and Irrigation Systems", "Sewage Treatment Facilities", "Steam and Air-Conditioning Supply", "New Single-Family Housing Construction (except For-Sale Builders)", "New Multifamily Housing Construction (except For-Sale Builders)", "New Housing For-Sale Builders", "Residential Remodelers", "Industrial Building Construction", "Commercial and Institutional Building Construction", "Water and Sewer Line and Related Structures Construction", "Oil and Gas Pipeline and Related Structures Construction", "Power and Communication Line and Related Structures Construction", "Land Subdivision", "Highway, Street, and Bridge Construction", "Other Heavy and Civil Engineering Construction", "Poured Concrete Foundation and Structure Contractors", "Structural Steel and Precast Concrete Contractors", "Framing Contractors", "Masonry Contractors", "Glass and Glazing Contractors", "Roofing Contractors", "Siding Contractors", "Other Foundation, Structure, and Building Exterior Contractors", "Electrical Contractors and Other Wiring Installation Contractors", "Plumbing, Heating, and Air-Conditioning Contractors", "Other Building Equipment Contractors", "Drywall and Insulation Contractors", "Painting and Wall Covering Contractors", "Flooring Contractors", "Tile and Terrazzo Contractors", "Finish Carpentry Contractors", "Other Building Finishing Contractors", "Site Preparation Contractors", "All Other Specialty Trade Contractors", "Dog and Cat Food Manufacturing", "Other Animal Food Manufacturing", "Flour Milling", "Rice Milling", "Malt Manufacturing", "Wet Corn Milling", "Soybean and Other Oilseed Processing", "Fats and Oils Refining and Blending", "Breakfast Cereal Manufacturing", "Beet Sugar Manufacturing", "Cane Sugar Manufacturing", "Nonchocolate Confectionery Manufacturing", "Chocolate and Confectionery Manufacturing from Cacao Beans", "Confectionery Manufacturing from Purchased Chocolate", "Frozen Fruit, Juice, and Vegetable Manufacturing", "Frozen Specialty Food Manufacturing", "Fruit and Vegetable Canning", "Specialty Canning", "Dried and Dehydrated Food Manufacturing", "Fluid Milk Manufacturing", "Creamery Butter Manufacturing", "Cheese Manufacturing", "Dry, Condensed, and Evaporated Dairy Product Manufacturing", "Ice Cream and Frozen Dessert Manufacturing", "Animal (except Poultry) Slaughtering", "Meat Processed from Carcasses", "Rendering and Meat Byproduct Processing", "Poultry Processing", "Seafood Product Preparation and Packaging", "Retail Bakeries", "Commercial Bakeries", "Frozen Cakes, Pies, and Other Pastries Manufacturing", "Cookie and Cracker Manufacturing", "Dry Pasta, Dough, and Flour Mixes Manufacturing from Purchased Flour", "Tortilla Manufacturing", "Roasted Nuts and Peanut Butter Manufacturing", "Other Snack Food Manufacturing", "Coffee and Tea Manufacturing", "Flavoring Syrup and Concentrate Manufacturing", "Mayonnaise, Dressing, and Other Prepared Sauce Manufacturing", "Spice and Extract Manufacturing", "Perishable Prepared Food Manufacturing", "All Other Miscellaneous Food Manufacturing", "Soft Drink Manufacturing", "Bottled Water Manufacturing", "Ice Manufacturing", "Breweries", "Wineries", "Distilleries", "Tobacco Manufacturing", "Fiber, Yarn, and Thread Mills", "Broadwoven Fabric Mills", "Narrow Fabric Mills and Schiffli Machine Embroidery", "Nonwoven Fabric Mills", "Knit Fabric Mills", "Textile and Fabric Finishing Mills", "Fabric Coating Mills", "Carpet and Rug Mills", "Curtain and Linen Mills", "Textile Bag and Canvas Mills", "Rope, Cordage, Twine, Tire Cord, and Tire Fabric Mills", "All Other Miscellaneous Textile Product Mills", "Hosiery and Sock Mills", "Other Apparel Knitting Mills", "Cut and Sew Apparel Contractors", "Men's and Boys' Cut and Sew Apparel Manufacturing", "Women's, Girls', and Infants' Cut and Sew Apparel Manufacturing", "Other Cut and Sew Apparel Manufacturing", "Apparel Accessories and Other Apparel Manufacturing", "Leather and Hide Tanning and Finishing", "Footwear Manufacturing", "Women's Handbag and Purse Manufacturing", "All Other Leather Good and Allied Product Manufacturing", "Sawmills", "Wood Preservation", "Hardwood Veneer and Plywood Manufacturing", "Softwood Veneer and Plywood Manufacturing", "Engineered Wood Member (except Truss) Manufacturing", "Truss Manufacturing", "Reconstituted Wood Product Manufacturing", "Wood Window and Door Manufacturing", "Cut Stock, Resawing Lumber, and Planing", "Other Millwork (including Flooring)", "Wood Container and Pallet Manufacturing", "Manufactured Home (Mobile Home) Manufacturing", "Prefabricated Wood Building Manufacturing", "All Other Miscellaneous Wood Product Manufacturing", "Pulp Mills", "Paper (except Newsprint) Mills", "Newsprint Mills", "Paperboard Mills", "Corrugated and Solid Fiber Box Manufacturing", "Folding Paperboard Box Manufacturing", "Other Paperboard Container Manufacturing", "Paper Bag and Coated and Treated Paper Manufacturing", "Stationery Product Manufacturing", "Sanitary Paper Product Manufacturing", "All Other Converted Paper Product Manufacturing", "Commercial Printing (except Screen and Books)", "Commercial Screen Printing", "Books Printing", "Support Activites for Printing", "Petroleum Refineries", "Asphalt Paving Mixture and Block Manufacturing", "Asphalt Shingle and Coating Materials Manufacturing", "Petroleum Lubricating Oil and Grease Manufacturing", "All Other Petroleum and Coal Products Manufacturing", "Petrochemical Manufacturing", "Industrial Gas Manufacturing", "Synthetic Dye and Pigment Manufacturing", "Other Basic Inorganic Chemical Manufacturing", "Ethyl Alcohol Manufacturing", "Cyclic Crude, Intermediate, and Gum and Wood Chemical Manufacturing", "All Other Basic Organic Chemical Manufacturing", "Plastics Material and Resin Manufacturing", "Synthetic Rubber Manufacturing", "Artificial and Synthetic Fibers and Filaments Manufacturing", "Nitrogenous Fertilizer Manufacturing", "Phosphatic Fertilizer Manufacturing", "Fertilizer (Mixing Only) Manufacturing", "Pesticide and Other Agricultural Chemical Manufacturing", "Medicinal and Botanical Manufacturing", "Pharmaceutical Preparation Manufacturing", "In-Vitro Diagnostic Substance Manufacturing", "Biological Product (except Diagnostic) Manufacturing", "Paint and Coating Manufacturing", "Adhesive Manufacturing", "Soap and Other Detergent Manufacturing", "Polish and Other Sanitation Good Manufacturing", "Surface Active Agent Manufacturing", "Toilet Preparation Manufacturing", "Printing Ink Manufacturing", "Explosives Manufacturing", "Custom Compounding of Purchased Resins", "Photographic Film, Paper, Plate, and Chemical Manufacturing", "All Other Miscellaneous Chemical Product and Preparation Manufacturing", "Plastics Bag and Pouch Manufacturing", "Plastics Packaging Film and Sheet (including Laminated) Manufacturing", "Unlaminated Plastics Film and Sheet (except Packaging) Manufacturing", "Unlaminated Plastics Profile Shape Manufacturing", "Plastics Pipe and Pipe Fitting Manufacturing", "Laminated Plastics Plate, Sheet (except Packaging), and Shape Manufacturing", "Polystyrene Foam Product Manufacturing", "Urethane and Other Foam Product (except Polystyrene) Manufacturing", "Plastics Bottle Manufacturing", "Plastics Plumbing Fixture Manufacturing", "All Other Plastics Product Manufacturing", "Tire Manufacturing (except Retreading)", "Tire Retreading", "Rubber and Plastics Hoses and Belting Manufacturing", "Rubber Product Manufacturing for Mechanical Use", "All Other Rubber Product Manufacturing", "Pottery, Ceramics, and Plumbing Fixture Manufacturing", "Clay Building Material and Refractories Manufacturing", "Flat Glass Manufacturing", "Other Pressed and Blown Glass and Glassware Manufacturing", "Glass Container Manufacturing", "Glass Product Manufacturing Made of Purchased Glass", "Cement Manufacturing", "Ready-Mix Concrete Manufacturing", "Concrete Block and Brick Manufacturing", "Concrete Pipe Manufacturing", "Other Concrete Product Manufacturing", "Lime Manufacturing", "Gypsum Product Manufacturing", "Abrasive Product Manufacturing", "Cut Stone and Stone Product Manufacturing", "Ground or Treated Mineral and Earth Manufacturing", "Mineral Wool Manufacturing", "All Other Miscellaneous Nonmetallic Mineral Product Manufacturing", "Iron and Steel Mills and Ferroalloy Manufacturing", "Iron and Steel Pipe and Tube Manufacturing from Purchased Steel", "Rolled Steel Shape Manufacturing", "Steel Wire Drawing", "Alumina Refining and Primary Aluminum Production", "Secondary Smelting and Alloying of Aluminum", "Aluminum Sheet, Plate, and Foil Manufacturing", "Other Aluminum Rolling, Drawing, and Extruding", "Nonferrous Metal (except Aluminum) Smelting and Refining", "Copper Rolling, Drawing, Extruding, and Alloying", "Nonferrous Metal (except Copper and Aluminum) Rolling, Drawing, and Extruding", "Secondary Smelting, Refining, and Alloying of Nonferrous Metal (except Copper and Aluminum)", "Iron Foundries", "Steel Investment Foundries", "Steel Foundries (except Investment)", "Nonferrous Metal Die-Casting Foundries", "Aluminum Foundries (except Die-Casting)", "Other Nonferrous Metal Foundries (except Die-Casting)", "Iron and Steel Forging", "Nonferrous Forging", "Custom Roll Forming", "Powder Metallurgy Part Manufacturing", "Metal Crown, Closure, and Other Metal Stamping (except Automotive)", "Metal Kitchen Cookware, Utensil, Cutlery, and Flatware (except Precious) Manufacturing", "Saw Blade and Handtool Manufacturing", "Prefabricated Metal Building and Component Manufacturing", "Fabricated Structural Metal Manufacturing", "Plate Work Manufacturing", "Metal Window and Door Manufacturing", "Sheet Metal Work Manufacturing", "Ornamental and Architectural Metal Work Manufacturing", "Power Boiler and Heat Exchanger Manufacturing", "Metal Tank (Heavy Gauge) Manufacturing", "Metal Can Manufacturing", "Other Metal Container Manufacturing", "Hardware Manufacturing", "Spring Manufacturing", "Other Fabricated Wire Product Manufacturing", "Machine Shops", "Precision Turned Product Manufacturing", "Bolt, Nut, Screw, Rivet, and Washer Manufacturing", "Metal Heat Treating", "Metal Coating, Engraving (except Jewelry and Silverware), and Allied Services to Manufacturers", "Electroplating, Plating, Polishing, Anodizing, and Coloring", "Industrial Valve Manufacturing", "Fluid Power Valve and Hose Fitting Manufacturing", "Plumbing Fixture Fitting and Trim Manufacturing", "Other Metal Valve and Pipe Fitting Manufacturing", "Ball and Roller Bearing Manufacturing", "Small Arms Ammunition Manufacturing", "Ammunition (except Small Arms) Manufacturing", "Small Arms, Ordnance, and Ordnance Accessories Manufacturing", "Fabricated Pipe and Pipe Fitting Manufacturing", "All Other Miscellaneous Fabricated Metal Product Manufacturing", "Farm Machinery and Equipment Manufacturing", "Lawn and Garden Tractor and Home Lawn and Garden Equipment Manufacturing", "Construction Machinery Manufacturing", "Mining Machinery and Equipment Manufacturing", "Oil and Gas Field Machinery and Equipment Manufacturing", "Food Product Machinery Manufacturing", "Semiconductor Machinery Manufacturing", "Sawmill, Woodworking, and Paper Machinery Manufacturing", "Printing Machinery and Equipment Manufacturing", "Other Industrial Machinery Manufacturing", "Optical Instrument and Lens Manufacturing", "Photographic and Photocopying Equipment Manufacturing", "Other Commercial and Service Industry Machinery Manufacturing", "Industrial and Commercial Fan and Blower and Air Purification Equipment Manufacturing", "Heating Equipment (except Warm Air Furnaces) Manufacturing", "Air-Conditioning and Warm Air Heating Equipment and Commercial and Industrial Refrigeration Equipment Manufacturing", "Industrial Mold Manufacturing", "Special Die and Tool, Die Set, Jig, and Fixture Manufacturing", "Cutting Tool and Machine Tool Accessory Manufacturing", "Machine Tool Manufacturing", "Rolling Mill and Other Metalworking Machinery Manufacturing", "Turbine and Turbine Generator Set Units Manufacturing", "Speed Changer, Industrial High-Speed Drive, and Gear Manufacturing", "Mechanical Power Transmission Equipment Manufacturing", "Other Engine Equipment Manufacturing", "Air and Gas Compressor Manufacturing", "Measuring, Dispensing, and Other Pumping Equipment Manufacturing", "Measuring, Dispensing, and Other Pumping Equipment Manufacturing", "Elevator and Moving Stairway Manufacturing", "Conveyor and Conveying Equipment Manufacturing", "Overhead Traveling Crane, Hoist, and Monorail System Manufacturing", "Industrial Truck, Tractor, Trailer, and Stacker Machinery Manufacturing", "Power-Driven Handtool Manufacturing", "Welding and Soldering Equipment Manufacturing", "Packaging Machinery Manufacturing", "Industrial Process Furnace and Oven Manufacturing", "Fluid Power Cylinder and Actuator Manufacturing", "Fluid Power Pump and Motor Manufacturing", "Scale and Balance Manufacturing", "All Other Miscellaneous General Purpose Machinery Manufacturing", "Electronic Computer Manufacturing", "Computer Storage Device Manufacturing", "Computer Terminal and Other Computer Peripheral Equipment Manufacturing", "Telephone Apparatus Manufacturing", "Radio and Television Broadcasting and Wireless Communications Equipment Manufacturing", "Other Communications Equipment Manufacturing", "Audio and Video Equipment Manufacturing", "Bare Printed Circuit Board Manufacturing", "Semiconductor and Related Device Manufacturing", "Capacitor, Resistor, Coil, Transformer, and Other Inductor Manufacturing", "Electronic Connector Manufacturing", "Printed Circuit Assembly (Electronic Assembly) Manufacturing", "Other Electronic Component Manufacturing", "Electromedical and Electrotherapeutic Apparatus Manufacturing", "Search, Detection, Navigation, Guidance, Aeronautical, and Nautical System and Instrument Manufacturing", "Automatic Environmental Control Manufacturing for Residential, Commercial, and Appliance Use", "Instruments and Related Products Manufacturing for Measuring, Displaying, and Controlling Industrial Process Variables", "Totalizing Fluid Meter and Counting Device Manufacturing", "Instrument Manufacturing for Measuring and Testing Electricity and Electrical Signals", "Analytical Laboratory Instrument Manufacturing", "Irradiation Apparatus Manufacturing", "Other Measuring and Controlling Device Manufacturing", "Blank Magnetic and Optical Recording Media Manufacturing", "Software and Other Prerecorded Compact Disc, Tape, and Record Reproducing", "Electric Lamp Bulb and Part Manufacturing", "Residential Electric Lighting Fixture Manufacturing", "Commercial, Industrial, and Institutional Electric Lighting Fixture Manufacturing", "Other Lighting Equipment Manufacturing", "Small Electrical Appliance Manufacturing", "Major Household Appliance Manufacturing", "Major Household Appliance Manufacturing", "Major Household Appliance Manufacturing", "Major Household Appliance Manufacturing", "Power, Distribution, and Specialty Transformer Manufacturing", "Motor and Generator Manufacturing", "Switchgear and Switchboard Apparatus Manufacturing", "Relay and Industrial Control Manufacturing", "Storage Battery Manufacturing", "Primary Battery Manufacturing", "Fiber Optic Cable Manufacturing", "Other Communication and Energy Wire Manufacturing", "Current-Carrying Wiring Device Manufacturing", "Noncurrent-Carrying Wiring Device Manufacturing", "Carbon and Graphite Product Manufacturing", "All Other Miscellaneous Electrical Equipment and Component Manufacturing", "Automobile Manufacturing", "Light Truck and Utility Vehicle Manufacturing", "Heavy Duty Truck Manufacturing", "Motor Vehicle Body Manufacturing", "Truck Trailer Manufacturing", "Motor Home Manufacturing", "Travel Trailer and Camper Manufacturing", "Motor Vehicle Gasoline Engine and Engine Parts Manufacturing", "Motor Vehicle Electrical and Electronic Equipment Manufacturing", "Motor Vehicle Steering and Suspension Components (except Spring) Manufacturing", "Motor Vehicle Brake System Manufacturing", "Motor Vehicle Transmission and Power Train Parts Manufacturing", "Motor Vehicle Seating and Interior Trim Manufacturing", "Motor Vehicle Metal Stamping", "Other Motor Vehicle Parts Manufacturing", "Aircraft Manufacturing", "Aircraft Engine and Engine Parts Manufacturing", "Other Aircraft Parts and Auxiliary Equipment Manufacturing", "Guided Missile and Space Vehicle Manufacturing", "Guided Missile and Space Vehicle Propulsion Unit and Propulsion Unit Parts Manufacturing", "Other Guided Missile and Space Vehicle Parts and Auxiliary Equipment Manufacturing", "Railroad Rolling Stock Manufacturing", "Ship Building and Repairing", "Boat Building", "Motorcycle, Bicycle, and Parts Manufacturing", "Military Armored Vehicle, Tank, and Tank Component Manufacturing", "All Other Transportation Equipment Manufacturing", "Wood Kitchen Cabinet and Countertop Manufacturing", "Upholstered Household Furniture Manufacturing", "Nonupholstered Wood Household Furniture Manufacturing", "Metal Household Furniture Manufacturing", "Household Furniture (except Wood and Metal) Manufacturing", "Institutional Furniture Manufacturing", "Wood Office Furniture Manufacturing", "Custom Architectural Woodwork and Millwork Manufacturing", "Office Furniture (except Wood) Manufacturing", "Showcase, Partition, Shelving, and Locker Manufacturing", "Mattress Manufacturing", "Blind and Shade Manufacturing", "Surgical and Medical Instrument Manufacturing", "Surgical Appliance and Supplies Manufacturing", "Dental Equipment and Supplies Manufacturing", "Ophthalmic Goods Manufacturing", "Dental Laboratories", "Jewelry and Silverware Manufacturing", "Sporting and Athletic Goods Manufacturing", "Doll, Toy, and Game Manufacturing", "Office Supplies (except Paper) Manufacturing", "Sign Manufacturing", "Gasket, Packing, and Sealing Device Manufacturing", "Musical Instrument Manufacturing", "Fastener, Button, Needle, and Pin Manufacturing", "Broom, Brush, and Mop Manufacturing", "Burial Casket Manufacturing", "All Other Miscellaneous Manufacturing", "Automobile and Other Motor Vehicle Merchant Wholesalers", "Motor Vehicle Supplies and New Parts Merchant Wholesalers", "Tire and Tube Merchant Wholesalers", "Motor Vehicle Parts (Used) Merchant Wholesalers", "Furniture Merchant Wholesalers", "Home Furnishing Merchant Wholesalers", "Lumber, Plywood, Millwork, and Wood Panel Merchant Wholesalers", "Brick, Stone, and Related Construction Material Merchant Wholesalers", "Roofing, Siding, and Insulation Material Merchant Wholesalers", "Other Construction Material Merchant Wholesalers", "Photographic Equipment and Supplies Merchant Wholesalers", "Office Equipment Merchant Wholesalers", "Computer and Computer Peripheral Equipment and Software Merchant Wholesalers", "Other Commercial Equipment Merchant Wholesalers", "Medical, Dental, and Hospital Equipment and Supplies Merchant Wholesalers", "Ophthalmic Goods Merchant Wholesalers", "Other Professional Equipment and Supplies Merchant Wholesalers", "Metal Service Centers and Other Metal Merchant Wholesalers", "Coal and Other Mineral and Ore Merchant Wholesalers", "Electrical Apparatus and Equipment, Wiring Supplies, and Related Equipment Merchant Wholesalers", "Household Appliances, Electric Housewares, and Consumer Electronics Merchant Wholesalers", "Other Electronic Parts and Equipment Merchant Wholesalers", "Hardware Merchant Wholesalers", "Plumbing and Heating Equipment and Supplies (Hydronics) Merchant Wholesalers", "Warm Air Heating and Air-Conditioning Equipment and Supplies Merchant Wholesalers", "Refrigeration Equipment and Supplies Merchant Wholesalers", "Construction and Mining (except Oil Well) Machinery and Equipment Merchant Wholesalers", "Farm and Garden Machinery and Equipment Merchant Wholesalers", "Industrial Machinery and Equipment Merchant Wholesalers", "Industrial Supplies Merchant Wholesalers", "Service Establishment Equipment and Supplies Merchant Wholesalers", "Transportation Equipment and Supplies (except Motor Vehicle) Merchant Wholesalers", "Sporting and Recreational Goods and Supplies Merchant Wholesalers", "Toy and Hobby Goods and Supplies Merchant Wholesalers", "Recyclable Material Merchant Wholesalers", "Jewelry, Watch, Precious Stone, and Precious Metal Merchant Wholesalers", "Other Miscellaneous Durable Goods Merchant Wholesalers", "Printing and Writing Paper Merchant Wholesalers", "Stationery and Office Supplies Merchant Wholesalers", "Industrial and Personal Service Paper Merchant Wholesalers", "Drugs and Druggists' Sundries Merchant Wholesalers", "Piece Goods, Notions, and Other Dry Goods Merchant Wholesalers", "Men's and Boys' Clothing and Furnishings Merchant Wholesalers", "Women's, Children's, and Infants' Clothing and Accessories Merchant Wholesalers", "Footwear Merchant Wholesalers", "General Line Grocery Merchant Wholesalers", "Packaged Frozen Food Merchant Wholesalers", "Dairy Product (except Dried or Canned) Merchant Wholesalers", "Poultry and Poultry Product Merchant Wholesalers", "Confectionery Merchant Wholesalers", "Fish and Seafood Merchant Wholesalers", "Meat and Meat Product Merchant Wholesalers", "Fresh Fruit and Vegetable Merchant Wholesalers", "Other Grocery and Related Products Merchant Wholesalers", "Grain and Field Bean Merchant Wholesalers", "Livestock Merchant Wholesalers", "Other Farm Product Raw Material Merchant Wholesalers", "Plastics Materials and Basic Forms and Shapes Merchant Wholesalers", "Other Chemical and Allied Products Merchant Wholesalers", "Petroleum Bulk Stations and Terminals", "Petroleum and Petroleum Products Merchant Wholesalers (except Bulk Stations and Terminals)", "Beer and Ale Merchant Wholesalers", "Wine and Distilled Alcoholic Beverage Merchant Wholesalers", "Farm Supplies Merchant Wholesalers", "Book, Periodical, and Newspaper Merchant Wholesalers", "Flower, Nursery Stock, and Florists' Supplies Merchant Wholesalers", "Tobacco and Tobacco Product Merchant Wholesalers", "Paint, Varnish, and Supplies Merchant Wholesalers", "Other Miscellaneous Nondurable Goods Merchant Wholesalers", "Business to Business Electronic Markets", "Wholesale Trade Agents and Brokers", "New Car Dealers", "Used Car Dealers", "Recreational Vehicle Dealers", "Boat Dealers", "Motorcycle, ATV, and All Other Motor Vehicle Dealers", "Automotive Parts and Accessories Stores", "Tire Dealers", "Furniture Stores", "Floor Covering Stores", "Window Treatment Stores", "All Other Home Furnishings Stores", "Household Appliance Stores", "Electronics Stores", "Home Centers", "Paint and Wallpaper Stores", "Hardware Stores", "Other Building Material Dealers", "Outdoor Power Equipment Stores", "Nursery, Garden Center, and Farm Supply Stores", "Supermarkets and Other Grocery (except Convenience) Stores", "Convenience Stores", "Meat Markets", "Fish and Seafood Markets", "Fruit and Vegetable Markets", "Baked Goods Stores", "Confectionery and Nut Stores", "All Other Specialty Food Stores", "Beer, Wine, and Liquor Stores", "Pharmacies and Drug Stores", "Cosmetics, Beauty Supplies, and Perfume Stores", "Optical Goods Stores", "Food (Health) Supplement Stores", "All Other Health and Personal Care Stores", "Gasoline Stations with Convenience Stores", "Other Gasoline Stations", "Men's Clothing Stores", "Women's Clothing Stores", "Children's and Infants' Clothing Stores", "Family Clothing Stores", "Clothing Accessories Stores", "Other Clothing Stores", "Shoe Stores", "Jewelry Stores", "Luggage and Leather Goods Stores", "Sporting Goods Stores", "Hobby, Toy, and Game Stores", "Sewing, Needlework, and Piece Goods Stores", "Musical Instrument and Supplies Stores", "Book Stores", "News Dealers and Newsstands", "Department Stores", "Department Stores", "Warehouse Clubs and Supercenters", "Warehouse Clubs and Supercenters", "All Other General Merchandise Stores", "Florists", "Office Supplies and Stationery Stores", "Gift, Novelty, and Souvenir Stores", "Used Merchandise Stores", "Pet and Pet Supplies Stores", "Art Dealers", "Manufactured (Mobile) Home Dealers", "Tobacco Stores", "All Other Miscellaneous Store Retailers (except Tobacco Stores)", "Electronic Shopping and Mail-Order Houses", "Electronic Shopping and Mail-Order Houses", "Electronic Shopping and Mail-Order Houses", "Vending Machine Operators", "Fuel Dealers", "Other Direct Selling Establishments", "Scheduled Passenger Air Transportation", "Scheduled Freight Air Transportation", "Nonscheduled Chartered Passenger Air Transportation", "Nonscheduled Chartered Freight Air Transportation", "Other Nonscheduled Air Transportation", "Line-Haul Railroads", "Short Line Railroads", "Deep Sea Freight Transportation", "Deep Sea Passenger Transportation", "Coastal and Great Lakes Freight Transportation", "Coastal and Great Lakes Passenger Transportation", "Inland Water Freight Transportation", "Inland Water Passenger Transportation", "General Freight Trucking, Local", "General Freight Trucking, Long-Distance, Truckload", "General Freight Trucking, Long-Distance, Less Than Truckload", "Used Household and Office Goods Moving", "Specialized Freight (except Used Goods) Trucking, Local", "Specialized Freight (except Used Goods) Trucking, Long-Distance", "Mixed Mode Transit Systems", "Commuter Rail Systems", "Bus and Other Motor Vehicle Transit Systems", "Other Urban Transit Systems", "Interurban and Rural Bus Transportation", "Taxi Service", "Limousine Service", "School and Employee Bus Transportation", "Charter Bus Industry", "Special Needs Transportation", "All Other Transit and Ground Passenger Transportation", "Pipeline Transportation of Crude Oil", "Pipeline Transportation of Natural Gas", "Pipeline Transportation of Refined Petroleum Products", "All Other Pipeline Transportation", "Scenic and Sightseeing Transportation, Land", "Scenic and Sightseeing Transportation, Water", "Scenic and Sightseeing Transportation, Other", "Air Traffic Control", "Other Airport Operations", "Other Support Activities for Air Transportation", "Support Activities for Rail Transportation", "Port and Harbor Operations", "Marine Cargo Handling", "Navigational Services to Shipping", "Other Support Activities for Water Transportation", "Motor Vehicle Towing", "Other Support Activities for Road Transportation", "Freight Transportation Arrangement", "Packing and Crating", "All Other Support Activities for Transportation", "Postal Service", "Couriers and Express Delivery Services", "Local Messengers and Local Delivery", "General Warehousing and Storage", "Refrigerated Warehousing and Storage", "Farm Product Warehousing and Storage", "Other Warehousing and Storage", "Newspaper Publishers", "Periodical Publishers", "Book Publishers", "Directory and Mailing List Publishers", "Greeting Card Publishers", "All Other Publishers", "Software Publishers", "Motion Picture and Video Production", "Motion Picture and Video Distribution", "Motion Picture Theaters (except Drive-Ins)", "Drive-In Motion Picture Theaters", "Teleproduction and Other Postproduction Services", "Other Motion Picture and Video Industries", "Music Publishers", "Sound Recording Studios", "Record Production and Distribution", "Record Production and Distribution", "Other Sound Recording Industries", "Radio Networks", "Radio Stations", "Television Broadcasting", "Cable and Other Subscription Programming", "Wired Telecommunications Carriers", "Wireless Telecommunications Carriers (except Satellite)", "Satellite Telecommunications", "Telecommunications Resellers", "All Other Telecommunications", "Data Processing, Hosting, and Related Services", "News Syndicates", "Libraries and Archives", "Internet Publishing and Broadcasting and Web Search Portals", "All Other Information Services", "Monetary Authorities-Central Bank", "Commercial Banking", "Savings Institutions", "Credit Unions", "Other Depository Credit Intermediation", "Credit Card Issuing", "Sales Financing", "Consumer Lending", "Real Estate Credit", "International Trade Financing", "Secondary Market Financing", "All Other Nondepository Credit Intermediation", "Mortgage and Nonmortgage Loan Brokers", "Financial Transactions Processing, Reserve, and Clearinghouse Activities", "Other Activities Related to Credit Intermediation", "Investment Banking and Securities Dealing", "Securities Brokerage", "Commodity Contracts Dealing", "Commodity Contracts Brokerage", "Securities and Commodity Exchanges", "Miscellaneous Intermediation", "Portfolio Management", "Investment Advice", "Trust, Fiduciary, and Custody Activities", "Miscellaneous Financial Investment Activities", "Direct Life Insurance Carriers", "Direct Health and Medical Insurance Carriers", "Direct Property and Casualty Insurance Carriers", "Direct Title Insurance Carriers", "Other Direct Insurance (except Life, Health, and Medical) Carriers", "Reinsurance Carriers", "Insurance Agencies and Brokerages", "Claims Adjusting", "Third Party Administration of Insurance and Pension Funds", "All Other Insurance Related Activities", "Pension Funds", "Health and Welfare Funds", "Other Insurance Funds", "Open-End Investment Funds", "Trusts, Estates, and Agency Accounts", "Other Financial Vehicles", "Lessors of Residential Buildings and Dwellings", "Lessors of Nonresidential Buildings (except Miniwarehouses)", "Lessors of Miniwarehouses and Self-Storage Units", "Lessors of Other Real Estate Property", "Offices of Real Estate Agents and Brokers", "Residential Property Managers", "Nonresidential Property Managers", "Offices of Real Estate Appraisers", "Other Activities Related to Real Estate", "Passenger Car Rental", "Passenger Car Leasing", "Truck, Utility Trailer, and RV (Recreational Vehicle) Rental and Leasing", "Consumer Electronics and Appliances Rental", "Formal Wear and Costume Rental", "Video Tape and Disc Rental", "Home Health Equipment Rental", "Recreational Goods Rental", "All Other Consumer Goods Rental", "General Rental Centers", "Commercial Air, Rail, and Water Transportation Equipment Rental and Leasing", "Construction, Mining, and Forestry Machinery and Equipment Rental and Leasing", "Office Machinery and Equipment Rental and Leasing", "Other Commercial and Industrial Machinery and Equipment Rental and Leasing", "Lessors of Nonfinancial Intangible Assets (except Copyrighted Works)", "Offices of Lawyers", "Offices of Notaries", "Title Abstract and Settlement Offices", "All Other Legal Services", "Offices of Certified Public Accountants", "Tax Preparation Services", "Payroll Services", "Other Accounting Services", "Architectural Services", "Landscape Architectural Services", "Engineering Services", "Drafting Services", "Building Inspection Services", "Geophysical Surveying and Mapping Services", "Surveying and Mapping (except Geophysical) Services", "Testing Laboratories", "Interior Design Services", "Industrial Design Services", "Graphic Design Services", "Other Specialized Design Services", "Custom Computer Programming Services", "Computer Systems Design Services", "Computer Facilities Management Services", "Other Computer Related Services", "Administrative Management and General Management Consulting Services", "Human Resources Consulting Services", "Marketing Consulting Services", "Process, Physical Distribution, and Logistics Consulting Services", "Other Management Consulting Services", "Environmental Consulting Services", "Other Scientific and Technical Consulting Services", "Research and Development in Nanotechnology", "Research and Development in Nanotechnology", "Research and Development in Biotechnology (except Nanobiotechnology)", "Research and Development in the Physical, Engineering, and Life Sciences (except Nanotechnology and Biotechnology)", "Research and Development in the Social Sciences and Humanities", "Advertising Agencies", "Public Relations Agencies", "Media Buying Agencies", "Media Representatives", "Outdoor Advertising", "Direct Mail Advertising", "Advertising Material Distribution Services", "Other Services Related to Advertising", "Marketing Research and Public Opinion Polling", "Photography Studios, Portrait", "Commercial Photography", "Translation and Interpretation Services", "Veterinary Services", "All Other Professional, Scientific, and Technical Services", "Offices of Bank Holding Companies", "Offices of Other Holding Companies", "Corporate, Subsidiary, and Regional Managing Offices", "Office Administrative Services", "Facilities Support Services", "Employment Placement Agencies", "Executive Search Services", "Temporary Help Services", "Professional Employer Organizations", "Document Preparation Services", "Telephone Answering Services", "Telemarketing Bureaus and Other Contact Centers", "Private Mail Centers", "Other Business Service Centers (including Copy Shops)", "Collection Agencies", "Credit Bureaus", "Repossession Services", "Court Reporting and Stenotype Services", "All Other Business Support Services", "Travel Agencies", "Tour Operators", "Convention and Visitors Bureaus", "All Other Travel Arrangement and Reservation Services", "Investigation Services", "Security Guards and Patrol Services", "Armored Car Services", "Security Systems Services (except Locksmiths)", "Locksmiths", "Exterminating and Pest Control Services", "Janitorial Services", "Landscaping Services", "Carpet and Upholstery Cleaning Services", "Other Services to Buildings and Dwellings", "Packaging and Labeling Services", "Convention and Trade Show Organizers", "All Other Support Services", "Solid Waste Collection", "Hazardous Waste Collection", "Other Waste Collection", "Hazardous Waste Treatment and Disposal", "Solid Waste Landfill", "Solid Waste Combustors and Incinerators", "Other Nonhazardous Waste Treatment and Disposal", "Remediation Services", "Materials Recovery Facilities", "Septic Tank and Related Services", "All Other Miscellaneous Waste Management Services", "Elementary and Secondary Schools", "Junior Colleges", "Colleges, Universities, and Professional Schools", "Business and Secretarial Schools", "Computer Training", "Professional and Management Development Training", "Cosmetology and Barber Schools", "Flight Training", "Apprenticeship Training", "Other Technical and Trade Schools", "Fine Arts Schools", "Sports and Recreation Instruction", "Language Schools", "Exam Preparation and Tutoring", "Automobile Driving Schools", "All Other Miscellaneous Schools and Instruction", "Educational Support Services", "Offices of Physicians (except Mental Health Specialists)", "Offices of Physicians, Mental Health Specialists", "Offices of Dentists", "Offices of Chiropractors", "Offices of Optometrists", "Offices of Mental Health Practitioners (except Physicians)", "Offices of Physical, Occupational and Speech Therapists, and Audiologists", "Offices of Podiatrists", "Offices of All Other Miscellaneous Health Practitioners", "Family Planning Centers", "Outpatient Mental Health and Substance Abuse Centers", "HMO Medical Centers", "Kidney Dialysis Centers", "Freestanding Ambulatory Surgical and Emergency Centers", "All Other Outpatient Care Centers", "Medical Laboratories", "Diagnostic Imaging Centers", "Home Health Care Services", "Ambulance Services", "Blood and Organ Banks", "All Other Miscellaneous Ambulatory Health Care Services", "General Medical and Surgical Hospitals", "Psychiatric and Substance Abuse Hospitals", "Specialty (except Psychiatric and Substance Abuse) Hospitals", "Nursing Care Facilities (Skilled Nursing Facilities)", "Residential Intellectual and Developmental Disability Facilities", "Residential Mental Health and Substance Abuse Facilities", "Continuing Care Retirement Communities", "Assisted Living Facilities for the Elderly", "Other Residential Care Facilities", "Child and Youth Services", "Services for the Elderly and Persons with Disabilities", "Other Individual and Family Services", "Community Food Services", "Temporary Shelters", "Other Community Housing Services", "Emergency and Other Relief Services", "Vocational Rehabilitation Services", "Child Day Care Services", "Theater Companies and Dinner Theaters", "Dance Companies", "Musical Groups and Artists", "Other Performing Arts Companies", "Sports Teams and Clubs", "Racetracks", "Other Spectator Sports", "Promoters of Performing Arts, Sports, and Similar Events with Facilities", "Promoters of Performing Arts, Sports, and Similar Events without Facilities", "Agents and Managers for Artists, Athletes, Entertainers, and Other Public Figures", "Independent Artists, Writers, and Performers", "Museums", "Historical Sites", "Zoos and Botanical Gardens", "Nature Parks and Other Similar Institutions", "Amusement and Theme Parks", "Amusement Arcades", "Casinos (except Casino Hotels)", "Other Gambling Industries", "Golf Courses and Country Clubs", "Skiing Facilities", "Marinas", "Fitness and Recreational Sports Centers", "Bowling Centers", "All Other Amusement and Recreation Industries", "Hotels (except Casino Hotels) and Motels", "Casino Hotels", "Bed-and-Breakfast Inns", "All Other Traveler Accommodation", "RV (Recreational Vehicle) Parks and Campgrounds", "Recreational and Vacation Camps (except Campgrounds)", "Rooming and Boarding Houses, Dormitories, and Workers' Camps", "Food Service Contractors", "Caterers", "Mobile Food Services", "Drinking Places (Alcoholic Beverages)", "Full-Service Restaurants", "Limited-Service Restaurants", "Cafeterias, Grill Buffets, and Buffets", "Snack and Nonalcoholic Beverage Bars", "General Automotive Repair", "Automotive Exhaust System Repair", "Automotive Transmission Repair", "Other Automotive Mechanical and Electrical Repair and Maintenance", "Automotive Body, Paint, and Interior Repair and Maintenance", "Automotive Glass Replacement Shops", "Automotive Oil Change and Lubrication Shops", "Car Washes", "All Other Automotive Repair and Maintenance", "Consumer Electronics Repair and Maintenance", "Computer and Office Machine Repair and Maintenance", "Communication Equipment Repair and Maintenance", "Other Electronic and Precision Equipment Repair and Maintenance", "Commercial and Industrial Machinery and Equipment (except Automotive and Electronic) Repair and Maintenance", "Home and Garden Equipment Repair and Maintenance", "Appliance Repair and Maintenance", "Reupholstery and Furniture Repair", "Footwear and Leather Goods Repair", "Other Personal and Household Goods Repair and Maintenance", "Barber Shops", "Beauty Salons", "Nail Salons", "Diet and Weight Reducing Centers", "Other Personal Care Services", "Funeral Homes and Funeral Services", "Cemeteries and Crematories", "Coin-Operated Laundries and Drycleaners", "Drycleaning and Laundry Services (except Coin-Operated)", "Linen Supply", "Industrial Launderers", "Pet Care (except Veterinary) Services", "Photofinishing Laboratories (except One-Hour)", "One-Hour Photofinishing", "Parking Lots and Garages", "All Other Personal Services", "Religious Organizations", "Grantmaking Foundations", "Voluntary Health Organizations", "Other Grantmaking and Giving Services", "Human Rights Organizations", "Environment, Conservation and Wildlife Organizations", "Other Social Advocacy Organizations", "Civic and Social Organizations", "Business Associations", "Professional Organizations", "Labor Unions and Similar Labor Organizations", "Political Organizations", "Other Similar Organizations (except Business, Professional, Labor, and Political Organizations)", "Private Households", "Executive Offices", "Legislative Bodies", "Public Finance Activities", "Executive and Legislative Offices, Combined", "American Indian and Alaska Native Tribal Governments", "Other General Government Support", "Courts", "Police Protection", "Legal Counsel and Prosecution", "Correctional Institutions", "Parole Offices and Probation Offices", "Fire Protection", "Other Justice, Public Order, and Safety Activities", "Administration of Education Programs", "Administration of Public Health Programs", "Administration of Human Resource Programs (except Education, Public Health, and Veterans' Affairs Programs)", "Administration of Veterans' Affairs", "Administration of Air and Water Resource and Solid Waste Management Programs", "Administration of Conservation Programs", "Administration of Housing Programs", "Administration of Urban Planning and Community and Rural Development", "Administration of General Economic Programs", "Regulation and Administration of Transportation Programs", "Regulation and Administration of Communications, Electric, Gas, and Other Utilities", "Regulation of Agricultural Marketing and Commodities", "Regulation, Licensing, and Inspection of Miscellaneous Commercial Sectors", "Space Research and Technology", "National Security", "International Affairs "] 

# get the NAICS code by description string
def getNAICSCode(descStr):
    selectedDescription = [code for code in NAICS if code['Description'] == descStr]
    return selectedDescription[0]['NAICSCode']

def checkDirs():
    output_dir = f'{WORKING_PATH}/Outputs'
    if os.path.isdir(output_dir)==False:
        os.mkdir(output_dir)
    temp_dir = f'{WORKING_PATH}/Temp'
    if os.path.isdir(temp_dir)==False:
        os.mkdir(temp_dir)
    return [output_dir, temp_dir]

def getAll_WP_CSVs():
    return glob.glob(f"{WORKING_PATH}/*.csv")

def getAll_Temp_CSV():
    return glob.glob(f"{temp_dir}/*.csv")

def concattedCSV():
    return pd.concat([pd.read_csv(f) for f in sp_list ])


In [ ]:
#@title Slice and download

output_dir, temp_dir = checkDirs()
# grab csvs in working path
joined_list = getAll_WP_CSVs()

for idx, file in enumerate(joined_list):
    df = pd.read_csv(file)
    if (selected_attribute != 'None' and naics_code_descriptions != 'None'):
        z= df[df[selected_attribute].isin(acceptable_values)] 
        z = z[z['NAICSCode'].isin([getNAICSCode(naics_code_descriptions)])]
    elif (selected_attribute != 'None' and naics_code_descriptions == 'None'):
        z = df[df[selected_attribute].isin(acceptable_values)]
    if (naics_code_descriptions != 'None' and selected_attribute == 'None'):
        z= df[df['NAICSCode'].isin([getNAICSCode(naics_code_descriptions)])]
    z.to_csv(f'{temp_dir}/{dataFrameName}_{idx}.csv', index=False, encoding='utf-8-sig')

#get temp csvs, combine into one
sp_list = getAll_Temp_CSV()
combined_csv = concattedCSV()

#export to csv
combined_csv.to_csv(f'{output_dir}/{dataFrameName}.csv', index=False, encoding='utf-8-sig')

#remove temp files
for file in sp_list:
    os.remove(file)

print(f'Data succesfully retrieved. Your file is available at {output_dir}/{dataFrameName}.csv')
